<a href="https://colab.research.google.com/github/NishaMDev/Poem-Analysis/blob/main/Poem_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **NLP on Poetry.**


## TEAM -- ML-Drivers

### **By: Nisha M Devadiga**
### **Student ID: 015922205**

### **Install Libraries**

In [ ]:
import os, sys
from google.colab import drive
#drive.mount('/content/mnt')
#nb_path = '/content/notebooks'
#os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
#sys.path.insert(0, nb_path)  # or append(nb_path)

In [ ]:
# Install only once. Tomorrow, you can skip this.
!pip install beautifulsoup4 pandas-profiling contractions nltk gensim transformers sentence-transformers pyLDAvis spacy

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
team_folder ="ML-Drivers/NLP-Poetry/Nisha/"
shared_drive_team ="/content/drive/MyDrive/ML-Spring-2022/"+ team_folder
import os
os.chdir(shared_drive_team)

In [ ]:
pwd

'/content/drive/.shortcut-targets-by-id/1CVe7u-1UZ-HqzNeoVWEI6rHzcmHcrHOb/ML-Spring-2022/ML-Drivers/NLP-Poetry/Nisha'

In [ ]:
pip install contractions  transformers sentence-transformers 

In [ ]:
import numpy as np
import pandas as pd
import bs4 as bs
import re
import os
import glob
import matplotlib.pyplot as plt
import re
import string
import wordcloud
import nltk
import unicodedata
import contractions
import pickle
import urllib.request
import requests
import spacy
import json
import random
import joblib
%matplotlib inline

from bs4 import BeautifulSoup
from bs4.element import Comment
from os.path import isfile
from IPython.display import HTML, display
from IPython.display import clear_output
from tqdm import tqdm

from collections import defaultdict
from contractions import contractions_dict

from pprint import pprint

from transformers import pipeline
from sentence_transformers import SentenceTransformer, util

from spacy import displacy
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
from nltk.tag import pos_tag
from nltk.tokenize.toktok import ToktokTokenizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

from sklearn.feature_extraction import text

import gensim 
import gensim.corpora as corpora
from gensim.models import Word2Vec 
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('wordnet')
nltk.download('punkt')

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Visualize the topics

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


tokenizer = ToktokTokenizer()


# Re-add the additional stop words since we are recreating the document-term matrix
#add_stop_words = ['©', 'owner' ,'provided' ,'at' ,'charge' , 'educational', 'purposes']

#stop_words = text.ENGLISH_STOP_WORDS.union(add_stop_words)

stop_words = nltk.corpus.stopwords.words('english')
stops = set(stop_words)
stemmer = nltk.stem.SnowballStemmer('english')
wordlemmatizer = WordNetLemmatizer()
tokenizer = nltk.word_tokenize

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


### **Selected 2 Poets from allpoetry.com**

### 1. Sylvia Plath 

An American poet, novelist, and short-story writer. She is credited with advancing the genre of confessional poetry and is best known for two of her published collections, The Colossus and Other Poems (1960) and Ariel (1965), as well as The Bell Jar, a semi-autobiographical novel published shortly before her death in 1963. 

***Sylvia Plath's early poems exhibit what became her typical imagery, using personal and nature-based depictions featuring, for example, the moon, blood, hospitals, fetuses, and skulls.*** Plath's landscape poetry, which she wrote throughout her life, has been described as **"a rich and important area of her work that is often overlooked, some of the best of which was written about the Yorkshire moors".** 

After 1960 her work moved into a more surreal landscape darkened by a sense of imprisonment and looming death, overshadowed by her father. The Colossus is shot through with themes of death, redemption and resurrection. After her husband left, Plath produced, in less than two months, the 40 poems of rage, despair, love, and vengeance on which her reputation mostly rests.


Some in the feminist movement saw Plath as speaking for their experience, as a "symbol of blighted female genius".


### 2. Rabindranath Tagore

A Indian Bengali polymath who worked as a poet, writer, playwright, composer, philosopher, social reformer and painter.He reshaped Bengali literature and music as well as Indian art with Contextual Modernism in the late 19th and early 20th centuries. Author of the "profoundly sensitive, fresh and beautiful" poetry of Gitanjali, he became in 1913 the first non-European and the first lyricist to win the Nobel Prize in Literature.

His verse (topic), short stories, and novels **were acclaimed their lyricism, colloquialism, naturalism, and unnatural contemplation.** His compositions were chosen by two nations as national anthems: India's "Jana Gana Mana" and Bangladesh's "Amar Shonar Bangla". The Sri Lankan national anthem was inspired by his work.

His poems took on a lyrical voice of the moner manush - "man within the heart" and Tagore's "life force of his deep recesses", or meditating upon the jeevan devata—the demiurge or the "living God within".This figure connected with divinity through appeal to nature and the emotional interplay of human drama. 

In [ ]:
Sylvia_Plath_poems = "Sylvia_Plath_new.txt"
Rabindranth_Tagore_poems = "Rabindranth_Tagore_new.txt"


### **Selected 10 poems from each poet.**

In [ ]:
def beautify_txt(text):
    final_data = (((text).replace(u'\xa0', u' ')).replace(r'\r',u'\n').replace(r'\n', ' ').replace('© by owner. provided at no charge for educational purposes ',' '))
    return final_data

Sylvia_Plath_Poems=["https://allpoetry.com/Childless-Woman",
                    "https://allpoetry.com/poem/8498423-Aftermath-by-Sylvia-Plath",
                    "https://allpoetry.com/poem/8498467-Ariel-by-Sylvia-Plath",
                    "https://allpoetry.com/Barren-Woman",
                    "https://allpoetry.com/Amnesiac",
                    "https://allpoetry.com/poem/8498495-Daddy-by-Sylvia-Platht",
                    "https://allpoetry.com/Suicide-Off-Egg-Rock",
                    "https://allpoetry.com/poem/8498497-Lady-Lazarus-by-Sylvia-Plath",
                    "https://allpoetry.com/Parliament-Hill-Fields",
                    "https://allpoetry.com/poem/8498491-Tulips-by-Sylvia-Plath"]

Rabindranth_Tagore_Poems =["https://allpoetry.com/poem/8516489-Fairyland-by-Rabindranath-Tagore",
                      "https://allpoetry.com/poem/8516487-Fool-by-Rabindranath-Tagore",
                     "https://allpoetry.com/poem/8516605-Fireflies-by-Rabindranath-Tagore",
                     "https://allpoetry.com/Authorship",
                     "https://allpoetry.com/poem/8516525-I-by-Rabindranath-Tagore",
                     "https://allpoetry.com/poem/8516541-Friend-by-Rabindranath-Tagore",
                     "https://allpoetry.com/Defamation",
                     "https://allpoetry.com/Endless-Time",
                     "https://allpoetry.com/I-Am-Restless",
                     "https://allpoetry.com/Clouds-And-Waves"
            ]


Sample function to print out a poem . Make sure your outputs of poems esp when you juxtapose the POS from different poets, are in human readable format!

### **Scrapping the Poems**

In [ ]:
# for scrapping poems

def scrapePoetsPoems(PoemList, result):
  for url in PoemList:
    header = {'User-Agent':'Mozilla/5.0'}
    requests = urllib.request.Request(url,headers=header)
    requestread = urllib.request.urlopen(requests).read()
    soup = bs.BeautifulSoup(requestread,'html.parser')
    

    poem_content = (beautify_txt(soup.find_all('div', class_="poem_body")[0].text))
    poem_title = soup.find_all('h1')[0].text
    result.append([poem_title, poem_content])
  return result

In [ ]:
result = []
poems_results_SP = scrapePoetsPoems(Sylvia_Plath_Poems, result)

In [ ]:
poems_results_SP

[['Childless Woman',
  "\r\n\tThe womb\r\n\tRattles its pod, the moon\r\n\tDischarges itself from the tree with nowhere to go.\r\n\tMy landscape is a hand with no lines,\r\n\tThe roads bunched to a knot,\r\n\tThe knot myself,\r\n\tMyself the rose you acheive—-\r\n\tThis body,\r\n\tThis ivory\r\n\tUngodly as a child's shriek.\r\n\tSpiderlike, I spin mirrors,\r\n\tLoyal to my image,\r\n\tUttering nothing but blood—-\r\n\tTaste it, dark red!\r\n\tAnd my forest\r\n\tMy funeral,\r\n\tAnd this hill and this\r\n\tGleaming with the mouths of corpses.      "],
 ['Aftermath',
  "Compelled by calamity's magnet \nThey loiter and stare as if the house \nBurnt-out were theirs, or as if they thought \nSome scandal might any minute ooze \nFrom a smoke-choked closet into light; \nNo deaths, no prodigious injuries \nGlut these hunters after an old meat, \nBlood-spoor of the austere tragedies. \n\nMother Medea in a green smock \nMoves humbly as any housewife through \nHer ruined apartments, taking stock 

### **Data set for Poet 1**

In [ ]:
CLEANR = re.compile(r'\r\n\t|\r|\n|\t')

def cleantext(rawtext):
  cleantext = re.sub(CLEANR, ' ', rawtext)
  return cleantext

In [ ]:
SP_DF = pd.DataFrame(poems_results_SP)
SP_DF.columns = ['Title', 'Poem']
SP_DF['Poet'] = "Sylvia Plath"

In [ ]:
SP_DF['Poem'] = SP_DF['Poem'].apply(cleantext)

In [ ]:
SP_DF

,Title,Poem,Poet
0,Childless Woman,"The womb Rattles its pod, the moon Discharges...",Sylvia Plath
1,Aftermath,Compelled by calamity's magnet They loiter an...,Sylvia Plath
2,Ariel,Stasis in darkness. Then the substanceless blu...,Sylvia Plath
3,Barren Woman,"Empty, I echo to the least footfall, Museum wi...",Sylvia Plath
4,Amnesiac,"No use, no use, now, begging Recognize!There i...",Sylvia Plath
5,Daddy,"You do not do, you do not do Any more, black ...",Sylvia Plath
6,Suicide Off Egg Rock,Behind him the hotdogs split and drizzled On t...,Sylvia Plath
7,Lady Lazarus,I have done it again. One year in every ten...,Sylvia Plath
8,Parliament Hill Fields,On this bald hill the new year hones its edge....,Sylvia Plath
9,Tulips,"The tulips are too excitable, it is winter her...",Sylvia Plath


### **Data set for Poet 2**

In [ ]:
result = []
poems_results_RT=scrapePoetsPoems(Rabindranth_Tagore_Poems,result)

In [ ]:
RT_DF = pd.DataFrame(poems_results_RT)
RT_DF.columns = ['Title', 'Poem']
RT_DF['Poet'] = "Rabindranath Tagore"

In [ ]:
RT_DF['Poem'] = RT_DF['Poem'].apply(cleantext)

In [ ]:
RT_DF

,Title,Poem,Poet
0,Fairyland,If people came to know where my king's palace ...,Rabindranath Tagore
1,Fool,"O Fool, try to carry thyself upon thy own shou...",Rabindranath Tagore
2,Fireflies,"My fancies are fireflies, — Specks of living ...",Rabindranath Tagore
3,Authorship,"You say that father write a lot of books, but ...",Rabindranath Tagore
4,I,I wonder if I know him In whose speech is my v...,Rabindranath Tagore
5,Friend,Art thou abroad on this stormy night on thy j...,Rabindranath Tagore
6,Defamation,"Where are those tears in your eyes, my child? ...",Rabindranath Tagore
7,Endless Time,"Time is endless in thy hands, my lord. There ...",Rabindranath Tagore
8,I Am Restless,I am restless. I am athirst for far-away thin...,Rabindranath Tagore
9,Clouds And Waves,"Mother, the folk who live up in the clouds cal...",Rabindranath Tagore


In [ ]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', 0)

In [ ]:
frames = [SP_DF,RT_DF]
data = pd.concat(frames,ignore_index=True)
data.head(10)

,Title,Poem,Poet
0,Childless Woman,"The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.",Sylvia Plath
1,Aftermath,"Compelled by calamity's magnet They loiter and stare as if the house Burnt-out were theirs, or as if they thought Some scandal might any minute ooze From a smoke-choked closet into light; No deaths, no prodigious injuries Glut these hunters after an old meat, Blood-spoor of the austere tragedies. Mother Medea in a green smock Moves humbly as any housewife through Her ruined apartments, taking stock Of charred shoes, the sodden upholstery: Cheated of the pyre and the rack, The crowd sucks her last tear and turns away.",Sylvia Plath
2,Ariel,"Stasis in darkness. Then the substanceless blue Pour of tor and distances. God's lioness, How one we grow, Pivot of heels and knees! —-The furrow Splits and passes, sister to The brown arc Of the neck I cannot catch, Nigger-eye Berries cast dark Hooks —- Black sweet blood mouthfuls, Shadows. Something else Hauls me through air —- Thighs, hair; Flakes from my heels. White Godiva, I unpeel —- Dead hands, dead stringencies. And now I Foam to wheat, a glitter of seas. The child's cry Melts in the wall. And I Am the arrow, The dew that flies, Suicidal, at one with the drive Into the red Eye, the cauldron of morning.",Sylvia Plath
3,Barren Woman,"Empty, I echo to the least footfall, Museum without statues, grand with pillars, porticoes, rotundas. In my courtyard a fountain leaps and sinks back into itself, Nun-hearted and blind to the world. Marble lilies Exhale their pallor like scent. I imagine myself with a great public, Mother of a white Nike and several bald-eyed Apollos. Instead, the dead injure me with attentions, and nothing can happen. Blank-faced and mum as a nurse.",Sylvia Plath
4,Amnesiac,"No use, no use, now, begging Recognize!There is nothing to do with such a beautiful blank but smooth it.Name, house, car keys,The little toy wife—Erased, sigh, sigh.Four babies and a cocker!Nurses the size of worms and a minute doctorTuck him in.Old happeningsPeel from his skin.Down the drain with all of it!Hugging his pillowLike the red-headed sister he never dared to touch,He dreams of a new one—Barren, the lot are barren!And of another color.How they'll travel, travel, travel, scenerySparking off their brother-sister rearsA comet tail!And money the sperm fluid of it all.One nurse brings inA green drink, one a blue.They rise on either side of him like stars.The two drinks flame and foam.O sister, mother, wife,Sweet Lethe is my life.I am never, never, never coming home!",Sylvia Plath
5,Daddy,"You do not do, you do not do Any more, black shoe In which I have lived like a foot For thirty years, poor and white, Barely daring to breathe or Achoo. Daddy, I have had to kill you. You died before I had time—- Marble-heavy, a bag full of God, Ghastly statue with one gray toe Big as a Frisco seal And a head in the freakish Atlantic Where it pours bean green over blue In the waters off the beautiful Nauset. I used to pray to recover you. Ach, du. In the German tongue, in the Polish town Scraped flat by the roller Of wars, wars, wars. But the name of the town is common. My Polack friend Says there are a dozen or two. So I never could tell where you Put your foot, your root, I never could talk to you. The tongue stuck in my jaw. It stuck in a barb wire snare. Ich, ich, ich, ich, I could hardly speak. I thought every German was you. And the language obscene An engine, an engine, Chuffing me off like a Jew. A Jew to Dachau, Auschwitz, Belsen. I began to talk like a Jew. I think I may well be a Jew. The snows of the Tyrol, the 

### Preparing the poem data and cleaning it.

In [ ]:
# removing special characters and stop words from the text

data['documents_cleaned']=data.Poem.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words) )


In [ ]:
 # tokenize the sentence and find the POS tag for each token
data['POS'] = data['documents_cleaned'].apply(nltk.word_tokenize)
data['POS'] = data['POS'].apply(nltk.pos_tag)

In [ ]:
data.head(2)

,Title,Poem,Poet,documents_cleaned,POS
0,Childless Woman,"The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.",Sylvia Plath,womb rattles pod moon discharges tree nowhere go landscape hand lines roads bunched knot knot myself rose acheive body ivory ungodly child s shriek spiderlike spin mirrors loyal image uttering nothing blood taste it dark red forest funeral hill gleaming mouths corpses,"[(womb, NN), (rattles, NNS), (pod, VBP), (moon, NN), (discharges, NNS), (tree, VBP), (nowhere, RB), (go, VB), (landscape, NN), (hand, NN), (lines, NNS), (roads, NNS), (bunched, VBD), (knot, JJ), (knot, NN), (myself, PRP), (rose, VBD), (acheive, JJ), (body, NN), (ivory, NN), (ungodly, RB), (child, JJ), (s, NN), (shriek, NN), (spiderlike, NN), (spin, NN), (mirrors, NNS), (loyal, VBP), (image, NN), (uttering, VBG), (nothing, NN), (blood, NN), (taste, NN), (it, PRP), (dark, VBZ), (red, JJ), (forest, JJ), (funeral, NN), (hill, NN), (gleaming, VBG), (mouths, NNS), (corpses, NNS)]"
1,Aftermath,"Compelled by calamity's magnet They loiter and stare as if the house Burnt-out were theirs, or as if they thought Some scandal might any minute ooze From a smoke-choked closet into light; No deaths, no prodigious injuries Glut these hunters after an old meat, Blood-spoor of the austere tragedies. Mother Medea in a green smock Moves humbly as any housewife through Her ruined apartments, taking stock Of charred shoes, the sodden upholstery: Cheated of the pyre and the rack, The crowd sucks her last tear and turns away.",Sylvia Plath,compelled calamity s magnet loiter stare house burnt out theirs thought scandal might minute ooze smoke choked closet light deaths prodigious injuries glut hunters old meat blood spoor austere tragedies mother medea green smock moves humbly housewife ruined apartments taking stock charred shoes sodden upholstery cheated pyre rack crowd sucks last tear turns away,"[(compelled, VBN), (calamity, NN), (s, JJ), (magnet, NN), (loiter, NN), (stare, NN), (house, NN), (burnt, VBD), (out, RP), (theirs, JJ), (thought, NN), (scandal, NN), (might, MD), (minute, VB), (ooze, JJ), (smoke, NN), (choked, VBD), (closet, NN), (light, JJ), (deaths, NNS), (prodigious, JJ), (injuries, NNS), (glut, VBP), (hunters, NNS), (old, JJ), (meat, NN), (blood, NN), (spoor, NN), (austere, JJ), (tragedies, NNS), (mother, VBP), (medea, NN), (green, JJ), (smock, NN), (moves, NNS), (humbly, RB), (housewife, VBP), (ruined, JJ), (apartments, NNS), (taking, VBG), (stock, NN), (charred, VBD), (shoes, NNS), (sodden, JJ), (upholstery, RB), (cheated, VBN), (pyre, JJ), (rack, NN), (crowd, NN), (sucks, NNS), (last, JJ), (tear, JJ), (turns, VBZ), (away, RB)]"


## **Writing to JSON File**

 ### **Convert dataset to CSV**

In [ ]:
Sylvia_Plath_poems = "Sylvia_Plath_new.csv"
Rabindranth_Tagore_poems = "Rabindranth_Tagore_new.csv"

In [ ]:
SP_DF.to_csv(Sylvia_Plath_poems, sep=';', index="False")
RT_DF.to_csv(Rabindranth_Tagore_poems, sep=';', index="False")

##**Identify POS tags for Both POETS and save them in JSON file**

In [ ]:

RT_DF['documents_cleaned']=RT_DF.Poem.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words) )
SP_DF['documents_cleaned']=SP_DF.Poem.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words) )


In [ ]:
 # tokenize the sentence and find the POS tag for each token
RT_DF['POS'] = RT_DF['documents_cleaned'].apply(nltk.word_tokenize)
RT_DF['POS'] = RT_DF['POS'].apply(nltk.pos_tag)

SP_DF['POS'] = SP_DF['documents_cleaned'].apply(nltk.word_tokenize)
SP_DF['POS'] = SP_DF['POS'].apply(nltk.pos_tag)

###**Identify the POS for POET 1.**

In [ ]:
RT_DF_pos = RT_DF.copy()
RT_DF_pos['verbs'] =''
RT_DF_pos['adjectives'] =''
RT_DF_pos['nouns'] =''
RT_DF_pos['adverbs'] =''

global_vrb = set()
global_nns = set()
global_adj = set()
global_adv = set()

for index, row in RT_DF_pos.iterrows():
  row = row.copy()
  pos_val = row['POS']
  vrb = set([word for (word,pos) in pos_val if (pos.startswith('VB'))])
  nns = set([word for (word,pos) in pos_val if (pos.startswith('NN'))])
  adj = set([word for (word,pos) in pos_val if (pos.startswith('JJ'))])
  adv = set([word for (word,pos) in pos_val if (pos.startswith('RB'))])

  RT_DF_pos.loc[index,'verbs'] = list(vrb)
  RT_DF_pos.loc[index,'adjectives'] = list(adj)
  RT_DF_pos.loc[index,'nouns'] = list(nns)
  RT_DF_pos.loc[index,'adverbs'] = list(adv)
  
  global_vrb = set.union(global_vrb, vrb)
  global_nns = set.union(global_nns, nns)
  global_adj = set.union(global_adj, adj)
  global_adv = set.union(global_adv, adv)

RT_DF_pos['all_verbs'] = [list(global_vrb) for i in RT_DF_pos.index]
RT_DF_pos['all_adjectives'] = [list(global_adj) for i in RT_DF_pos.index]
RT_DF_pos['all_nouns'] = [list(global_nns) for i in RT_DF_pos.index]
RT_DF_pos['all_adverbs'] = [list(global_adv) for i in RT_DF_pos.index]

###**Identify the POS for POET 2.**

In [ ]:
SP_DF_pos = SP_DF.copy()
SP_DF_pos['verbs'] =''
SP_DF_pos['adjectives'] =''
SP_DF_pos['nouns'] =''
SP_DF_pos['adverbs'] =''

global_vrb = set()
global_nns = set()
global_adj = set()
global_adv = set()

for index, row in SP_DF_pos.iterrows():
  row = row.copy()
  pos_val = row['POS']
  vrb = set([word for (word,pos) in pos_val if (pos.startswith('VB'))])
  nns = set([word for (word,pos) in pos_val if (pos.startswith('NN'))])
  adj = set([word for (word,pos) in pos_val if (pos.startswith('JJ'))])
  adv = set([word for (word,pos) in pos_val if (pos.startswith('RB'))])

  SP_DF_pos.loc[index,'verbs'] = list(vrb)
  SP_DF_pos.loc[index,'adjectives'] = list(adj)
  SP_DF_pos.loc[index,'nouns'] = list(nns)
  SP_DF_pos.loc[index,'adverbs'] = list(adv)
  
  global_vrb = set.union(global_vrb, vrb)
  global_nns = set.union(global_nns, nns)
  global_adj = set.union(global_adj, adj)
  global_adv = set.union(global_adv, adv)

SP_DF_pos['all_verbs'] = [list(global_vrb) for i in SP_DF_pos.index]
SP_DF_pos['all_adjectives'] = [list(global_adj) for i in SP_DF_pos.index]
SP_DF_pos['all_nouns'] = [list(global_nns) for i in SP_DF_pos.index]
SP_DF_pos['all_adverbs'] = [list(global_adv) for i in SP_DF_pos.index]

### **Convert To Json**

In [ ]:
# convert first poet dataset to json 
SP_json = SP_DF.to_json(orient = "index")
parse_json_sylvia = json.loads(SP_json)
json.dumps(parse_json_sylvia, indent= 2)

'{\n  "0": {\n    "Title": "Childless Woman",\n    "Poem": " The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive\\u2014- This body, This ivory Ungodly as a child\'s shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood\\u2014- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.      ",\n    "Poet": "Sylvia Plath",\n    "documents_cleaned": "womb rattles pod  moon discharges tree nowhere go  landscape hand lines  roads bunched knot  knot myself  rose acheive   body  ivory ungodly child s shriek  spiderlike  spin mirrors  loyal image  uttering nothing blood   taste it  dark red  forest funeral  hill gleaming mouths corpses ",\n    "POS": [\n      [\n        "womb",\n        "NN"\n      ],\n      [\n        "rattles",\n        "NNS"\n      ],\n      [\n        "po

In [ ]:
# convert second poet dataset to json 
RT_json = RT_DF.to_json(orient = "index")
parse_json_rabindranath = json.loads(RT_json)
json.dumps(parse_json_rabindranath, indent= 2)

'{\n  "0": {\n    "Title": "Fairyland",\n    "Poem": "If people came to know where my king\'s palace is, it would vanish  into the air.      The walls are of white silver and the roof of shining gold.      The queen lives in a palace with seven courtyards, and she wears a jewel that cost all the wealth of seven kingdoms.      But let me tell you, mother, in a whisper, where my king\'s palace is.      It is at the corner of our terrace where the pot of the tulsi plant stands.      The princess lies sleeping on the far-away shore of the seven impassable seas.      There is none in the world who can find her but myself.      She has bracelets on her arms and pearl drops in her ears; her hair sweeps down upon the floor.      She will wake when I touch her with my magic wand and jewels will fall from her lips when she smiles.      But let me whisper in your ear, mother; she is there in the corner of our terrace where the pot of the tulsi plant stands.      When it is time for you to go to t

In [ ]:
# To write in JSON file  - Slyvia Plath
df = SP_DF_pos[['Poet','Title','Poem','verbs','adjectives','nouns','adverbs','all_verbs','all_adjectives','all_nouns','all_adverbs']]

result = df.to_json(orient="split")
with open("Slyvia_Plath_poems_pos.json", "w") as outfile:
    outfile.write(result)

# To write in JSON file  - Rabindranath Tagore
df = RT_DF_pos[['Poet','Title','Poem','verbs','adjectives','nouns','adverbs','all_verbs','all_adjectives','all_nouns','all_adverbs']]

result = df.to_json(orient="split")
with open("Rabindranath_Tagore_poems_pos.json", "w") as outfile:
    outfile.write(result)

## **Comparison of poems using various embedding techniques.**

## Analyzing the poems as it is before swapping POS of one poet with another.

### TFidVector

In [ ]:
tfidfvectoriser=TfidfVectorizer(max_features=64)
tfidfvectoriser.fit(data.documents_cleaned)
tfidf_vectors=tfidfvectoriser.transform(data.documents_cleaned)

In [ ]:
tfidf_vectors.shape

(20, 64)

In [ ]:
tfidf_vectors=tfidf_vectors.toarray()
print (tfidf_vectors[0])

[0.         0.         0.         0.         0.         0.32843238
 0.         0.         0.4005203  0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.34937311 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.37311374 0.
 0.         0.         0.         0.32843238 0.         0.
 0.         0.37311374 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.47260823 0.
 0.         0.         0.         0.        ]


In [ ]:
# Every vector is already normalised to have unit L2 norm
np.linalg.norm(tfidf_vectors[0],ord=2)

1.0

Every document has been converted into a 64 dimensional vector. As we set the max_features=64

### Pairwise similarity

Pairwise cosine similarity would just be the dot product of the vectors becasue tf-idf vectors from sklearn are already normalized and L2 norm of these vectors is 1. So denominator of cosine similiarity formula is 1 in this case.


In [ ]:
pairwise_similarities=np.dot(tfidf_vectors,tfidf_vectors.T)
pairwise_differences=euclidean_distances(tfidf_vectors)

In [ ]:
print (tfidf_vectors[0])
print("**********************************************************")
print('\n')
print ("Pairwise Similarities Shape",pairwise_similarities.shape)
print("**********************************************************")
print('\n')
print ("Pairwise_Similarities[0][:]",pairwise_similarities[0][:])

[0.         0.         0.         0.         0.         0.32843238
 0.         0.         0.4005203  0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.34937311 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.37311374 0.
 0.         0.         0.         0.32843238 0.         0.
 0.         0.37311374 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.47260823 0.
 0.         0.         0.         0.        ]
**********************************************************


Pairwise Similarities Shape (20, 20)
**********************************************************


Pairwise_Similarities[0][:] [1.         0.         0.24330063 0.10944149 0.20991199 0.02204654
 0.         0.26516563 0.25051109 0.23868865 0.         0.
 0.15918514 0.05834865 0.07720377 0.0528684

similarity is highest, 1 at index 0 because they are the same documents

In [ ]:
def most_similar(doc_id,similarity_matrix,matrix):
    print (f'Poet: {data.iloc[doc_id]["Poet"]}')
    print (f'Poem: {data.iloc[doc_id]["Poem"]}')
    print ('\n')
    print (f'Similar Documents using {matrix}:')
    if matrix=='Cosine Similarity':
        similar_ix=np.argsort(similarity_matrix[doc_id])[::-1]
    elif matrix=='Euclidean Distance':
        similar_ix=np.argsort(similarity_matrix[doc_id])
    for ix in similar_ix:
        if ix==doc_id:
            continue
        print('\n')
        print("**********************************************************")
        print('\n')
        print (f'Poet: {data.iloc[ix]["Poet"]}')
        print (f'Poem: {data.iloc[ix]["Title"]}')
        print (f'Poem: {data.iloc[ix]["Poem"]}')
        print('----------------------------------------------')
        print (f'{matrix} : {similarity_matrix[doc_id][ix]}')
        print('----------------------------------------------')

In [ ]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Poet: Sylvia Plath
Poem:  The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.      


Similar Documents using Cosine Similarity:


**********************************************************


Poet: Sylvia Plath
Poem: Lady Lazarus
Poem:  I have done it again.   One year in every ten   I manage it——     A sort of walking miracle, my skin   Bright as a Nazi lampshade,   My right foot     A paperweight,   My face a featureless, fine   Jew linen.     Peel off the napkin   0 my enemy.   Do I terrify?——     The nose, the eye pits, the full set of teeth?   The sour breath   Will vanish in a day.     Soon, soon the fles

We have compared the 1st poem of Sylvia Plaths with another poet and notice that these two poets are very dissimilar showing the cosine similarity = 0 .

One can also observe that the similarity between Sylvia's poems are also very less.

In [ ]:
most_similar(0,pairwise_differences,'Euclidean Distance')  

Poet: Sylvia Plath
Poem:  The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.      


Similar Documents using Euclidean Distance:


**********************************************************


Poet: Sylvia Plath
Poem: Lady Lazarus
Poem:  I have done it again.   One year in every ten   I manage it——     A sort of walking miracle, my skin   Bright as a Nazi lampshade,   My right foot     A paperweight,   My face a featureless, fine   Jew linen.     Peel off the napkin   0 my enemy.   Do I terrify?——     The nose, the eye pits, the full set of teeth?   The sour breath   Will vanish in a day.     Soon, soon the fle

Similar trend is also observed when uaing Euclidean distance for comparing tow poems. 

### Embedding using Doc2vec model

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(data.documents_cleaned)]

In [ ]:
model_d2v = Doc2Vec(vector_size=100,alpha=0.025, min_count=1)
  
model_d2v.build_vocab(tagged_data)

for epoch in range(100):
    model_d2v.train(tagged_data,
                total_examples=model_d2v.corpus_count,
                epochs=model_d2v.epochs)

In [ ]:
document_embeddings=np.zeros((data.shape[0],100))

for i in range(len(document_embeddings)):
    document_embeddings[i]=model_d2v.docvecs[i]

In [ ]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [ ]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Poet: Sylvia Plath
Poem:  The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.      


Similar Documents using Cosine Similarity:


**********************************************************


Poet: Sylvia Plath
Poem: Ariel
Poem: Stasis in darkness. Then the substanceless blue Pour of tor and distances.  God's lioness, How one we grow, Pivot of heels and knees! —-The furrow  Splits and passes, sister to The brown arc Of the neck I cannot catch,  Nigger-eye Berries cast dark Hooks —-  Black sweet blood mouthfuls, Shadows. Something else  Hauls me through air —- Thighs, hair; Flakes from my heels.  White Godiva, I

An interesting observation using Doc2Vec model is that this model is capable of learning the similarities in a better way. This could be due to the fact that it takes word order into account, generalizes to longer documents, and can learn from unlabelled data. 

Here, is Poem 0 is most similar to (Poet: Rabindranath Tagore | Poem: Fireflies)

In [ ]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Poet: Sylvia Plath
Poem:  The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.      


Similar Documents using Euclidean Distance:


**********************************************************


Poet: Sylvia Plath
Poem: Ariel
Poem: Stasis in darkness. Then the substanceless blue Pour of tor and distances.  God's lioness, How one we grow, Pivot of heels and knees! —-The furrow  Splits and passes, sister to The brown arc Of the neck I cannot catch,  Nigger-eye Berries cast dark Hooks —-  Black sweet blood mouthfuls, Shadows. Something else  Hauls me through air —- Thighs, hair; Flakes from my heels.  White Godiva, 

Higher the Euclidean distance means, more dissimilar the two poems are.

Here, is Poem 0 is most similar to {Poet: Rabindranath Tagore | Poem: Friend }

### Embedding using BERT model

**Bidirectional Encoder Representations from Transformers (BERT).** 

Due to its highly pragmatic approach, and higher performance, BERT is highlighted for achieving state-of-the-art performance in many NLP tasks.

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
document_embeddings = sbert_model.encode(data['documents_cleaned'])

In [ ]:
pairwise_similarities=cosine_similarity(document_embeddings)
pairwise_differences=euclidean_distances(document_embeddings)

In [ ]:
most_similar(0,pairwise_similarities,'Cosine Similarity')

Poet: Sylvia Plath
Poem:  The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.      


Similar Documents using Cosine Similarity:


**********************************************************


Poet: Sylvia Plath
Poem: Ariel
Poem: Stasis in darkness. Then the substanceless blue Pour of tor and distances.  God's lioness, How one we grow, Pivot of heels and knees! —-The furrow  Splits and passes, sister to The brown arc Of the neck I cannot catch,  Nigger-eye Berries cast dark Hooks —-  Black sweet blood mouthfuls, Shadows. Something else  Hauls me through air —- Thighs, hair; Flakes from my heels.  White Godiva, I

BERT has an advantage over models like Word2Vec because while each word has a fixed representation under Word2Vec regardless of the context within which the word appears, BERT produces word representations that are dynamically informed by the words around them. 


Aside from capturing obvious differences like polysemy, the context-informed word embeddings capture other forms of information that result in more accurate feature representations, which in turn results in better model performance.

Here, is Poem 0 is most similar to {Poet: Rabindranath Tagore | Poem: Friend }. Also before that, the poems of Sylvia shows similarity with her other notable works like - Ariel, Parliament Hill Fields,Suicide Off Egg Rock and Amnesiec

In [ ]:
most_similar(0,pairwise_differences,'Euclidean Distance')

Poet: Sylvia Plath
Poem:  The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.      


Similar Documents using Euclidean Distance:


**********************************************************


Poet: Sylvia Plath
Poem: Ariel
Poem: Stasis in darkness. Then the substanceless blue Pour of tor and distances.  God's lioness, How one we grow, Pivot of heels and knees! —-The furrow  Splits and passes, sister to The brown arc Of the neck I cannot catch,  Nigger-eye Berries cast dark Hooks —-  Black sweet blood mouthfuls, Shadows. Something else  Hauls me through air —- Thighs, hair; Flakes from my heels.  White Godiva, 

Here as well, Poem 0 is most similar to {Poet: Rabindranath Tagore | Poem: Friend }. Also before that, the poems of Sylvia shows similarity with her other notable works like - Ariel, Parliament Hill Fields,Suicide Off Egg Rock and Amnesiec. Similar observation was also noted usng cosine similarities

##**Playing Defense by reading the poems from the file which was earlier saved in the shared drive.**  


###Read the poems from the shared drive and load it.

In [ ]:
#------------------------------------------
# try to get the poem, if it is there, load it 
# if not scarpe it into a df. #playdefense
#
#------------------------------------------
def get_poems(poets_poems):
  file_exists = os.path.exists(poets_poems)
  print(f"{poets_poems} file_exists")
  if (file_exists):
    #load it!
    df = pd.read_csv(poets_poems, sep=';')
    print(df.head(2))
    return df

In [ ]:
SP_DF1 = get_poems(Sylvia_Plath_poems)
RT_DF1 = get_poems(Rabindranth_Tagore_poems)

Sylvia_Plath_new.csv file_exists
   Unnamed: 0  ...          Poet
0  0           ...  Sylvia Plath
1  1           ...  Sylvia Plath

[2 rows x 4 columns]
Rabindranth_Tagore_new.csv file_exists
   Unnamed: 0  ...                 Poet
0  0           ...  Rabindranath Tagore
1  1           ...  Rabindranath Tagore

[2 rows x 4 columns]


In [ ]:
SP_DF1['Poem'][1]

"Compelled by calamity's magnet  They loiter and stare as if the house  Burnt-out were theirs, or as if they thought  Some scandal might any minute ooze  From a smoke-choked closet into light;  No deaths, no prodigious injuries  Glut these hunters after an old meat,  Blood-spoor of the austere tragedies.   Mother Medea in a green smock  Moves humbly as any housewife through  Her ruined apartments, taking stock  Of charred shoes, the sodden upholstery:  Cheated of the pyre and the rack,  The crowd sucks her last tear and turns away.      "

In [ ]:
RT_DF1['Poem'][1]

'O Fool, try to carry thyself upon thy own shoulders!  O beggar, to come beg at thy own door!   Leave all thy burdens on his hands who can bear all,  and never look behind in regret.   Thy desire at once puts out the light from the lamp it touches with its breath.  It is unholy—-take not thy gifts through its unclean hands.  Accept only what is offered by sacred love.      '

Sample function to print out a poem . Make sure your outputs of poems esp when you juxtapose the POS from different poets, are in human readable format!

In [ ]:
def print_the_poem(poet_df):
  #add iterrows to print all poems 04-8-2022 11:56 pm!
  poem = poet_df['Poem'][3].split('\n')
  for line in poem:
    print(f"{line}")

print_the_poem(SP_DF1)

Empty, I echo to the least footfall, Museum without statues, grand with pillars, porticoes, rotundas. In my courtyard a fountain leaps and sinks back into itself, Nun-hearted and blind to the world. Marble lilies Exhale their pallor like scent.  I imagine myself with a great public, Mother of a white Nike and several bald-eyed Apollos. Instead, the dead injure me with attentions, and nothing can happen. Blank-faced and mum as a nurse.      


## **Data Cleaning**

In [ ]:
def cleanPoemText(text):
    
    text = contractions.fix(text)
    
    text = text.strip().lower()
    
    text = re.sub(r"'", ' ', text)
    
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((i, " ") for i in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    text = ' '.join([w for w in text.split()])

    # Replace multiple space with one space
    text = re.sub(' +', ' ', text)
    
    text = ''.join(text)

    return text

## **Clean Poem Text**

In [ ]:
embedding = ''
def buildPoemArray(poet, data):
  poems = []
  for i in data:
    poem = data[i]["Poem"]

    clean_poem = cleanPoemText(poem)
    idx = "poem" + str(i)
    poet[idx] = clean_poem
    poems.append(idx)
  poet['poems_array'] = poems

In [ ]:
Sylvia_Plath = {

}

Rabindranth_Tagore = {

}

SP_json = SP_DF1.to_json(orient = "index")
parse_json_sylvia = json.loads(SP_json)

RT_json = RT_DF1.to_json(orient = "index")
parse_json_rabindranath = json.loads(RT_json)

buildPoemArray(Sylvia_Plath, parse_json_sylvia)
buildPoemArray(Rabindranth_Tagore, parse_json_rabindranath)

In [ ]:
Sylvia_Plath

{'poem0': 'the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses',
 'poem1': 'compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away',
 'poem2': 'stasis in darkness then the subs

In [ ]:
Rabindranth_Tagore

{'poem0': 'if people came to know where my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper where my king s palace is it is at the corner of our terrace where the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas there is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is there in the corner of our terrace where the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner where the shadow of the walls meet 

In [ ]:
Sylvia_Plath_poems = Sylvia_Plath.copy()
Rabindranth_Tagore_poems = Rabindranth_Tagore.copy()

## **Run Part-Of-Speech, extract verbs, nouns , adverbs and adjectives**

In [ ]:
def getPOS(poet):
  global_vrb= set()
  global_nns= set()
  global_adj= set()
  global_adv= set()

  for key in poet['poems_array']:
    text = poet[key]
    wrd_list = word_tokenize(text)
    pos_val = nltk.pos_tag(wrd_list)
    poet["pos_"+key] = pos_val

    vrb = set([word for (word, pos) in pos_val if (pos.startswith('VB'))])
    nns = set([word for (word, pos) in pos_val if (pos.startswith('NN'))])
    adj = set([word for (word, pos) in pos_val if (pos.startswith('JJ'))])
    adv = set([word for (word, pos) in pos_val if (pos.startswith('RB'))])

    poet["verbs_"+key] = list(vrb)
    poet["adjectives_"+key] = list(adj)
    poet["nouns_"+key] = list(nns)
    poet["adverbs_"+key] = list(adv)

    global_vrb = set.union(global_vrb, vrb)
    global_nns = set.union(global_nns, nns)
    global_adj = set.union(global_adj, adj)
    global_adv = set.union(global_adv, adv)

  poet['all_verbs'] = list(global_vrb)
  poet['all_nouns'] = list(global_nns)
  poet['all_adjectives'] = list(global_adj)
  poet['all_adverbs'] = list(global_adv)

In [ ]:
Sylvia_Plath_pos = Sylvia_Plath.copy()
getPOS(Sylvia_Plath_pos)

In [ ]:
Rabindranth_Tagore_pos = Rabindranth_Tagore.copy()
getPOS(Rabindranth_Tagore_pos)

## **Store the POS JSON Data to File**

In [ ]:
with open('Sylvia_Plath_pos_final.json', 'w') as outfile:
    json.dump(Sylvia_Plath_pos, outfile)

In [ ]:
with open('Rabindranth_Tagore_pos_final.json', 'w') as outfile:
    json.dump(Rabindranth_Tagore_pos, outfile)

## **Interchange the POS for both the poets using transformer**

When  the cosine similarity score is more than 0.3 then swap the POS tags.

In [ ]:
model = SentenceTransformer('stsb-roberta-large')

In [ ]:
def getSemanticSimilarity(p1,p2):
  embedding1 = model.encode(p1, convert_to_tensor=True)
  embedding2 = model.encode(p2, convert_to_tensor=True)
  cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
  print(cosine_scores) 
  similarity_score = cosine_scores.item()
  return similarity_score

In [ ]:
getSemanticSimilarity(Sylvia_Plath_pos['all_verbs'][0],Rabindranth_Tagore_pos["all_verbs"][0])

tensor([[0.1307]])


0.130704864859581

### **Function to Swap the Verbs based on similarity score**

In [ ]:
def transposeVerbs(p1,p2,text):
    for v1 in p1:
      for v2 in p2:
        score = getSemanticSimilarity(v1,v2)
        if score > 0.3:
          text = text.replace(v1,v2)
          print("verb")
          print(text)
          break
    return text

### **Function to Swap the Adjectives based on similarity score**

In [ ]:
def transposeAdjectives(p1,p2, poem):
  text = poem
  for a1 in p1:
    for a2 in p2:
      score = getSemanticSimilarity(a1,a2)
      if score > 0.3:
        text = text.replace(a1,a2)
        print("adjective")
        print(text)
        break
  return text

### **Function to Swap Nouns based on similarity score.** 

In [ ]:
def transposeNoun(p1,p2,text):
  for n1 in p1:
    for n2 in p2:
      score = getSemanticSimilarity(n1,n2)
      if score > 0.3:
        text  = text.replace(n1,n2)
        print("nouns")
        print(text)
        break
  return text     

### **Function to Swap the Adverbs based on similarity score**

In [ ]:
def transposeAdverbs(p1,p2, poem):
  text = poem
  for a1 in p1:
    for a2 in p2:
      score = getSemanticSimilarity(a1,a2)
      if score > 0.3:
        text = text.replace(a1,a2)
        print("adverbs")
        print(text)
        break
  return text

###**Function to TRANSPOSE/SUBSTITUTE** 

In [ ]:
def swapIteration(poet1, poet2, text):
    p2_verbs = poet2["all_verbs"]
    p1_verbs = poet1["all_verbs"]
    p2_verbs = p2_verbs[0:2]
    p1_verbs = p1_verbs[0:2]
    
    print(p1_verbs)
    print(p2_verbs)   
    new_text_v = transposeVerbs(p1_verbs, p2_verbs, text)
    print("--------------new text ( after substituting verbs )-------")
    print(new_text_v)
    # print(text)

    p2_adjectives = poet2["all_adjectives"]
    p1_adjectives = poet1["all_adjectives"]
    p2_adjectives = p2_adjectives[0:2]
    p1_adjectives = p1_adjectives[0:2]
    new_text_adj = transposeAdjectives(p1_adjectives, p2_adjectives, new_text_v)
    print("--------------new text ( after substituting adjectives )-------")
    print(new_text_adj)

    p2_nouns = poet2["all_nouns"]
    p1_nouns = poet1["all_nouns"]
    p2_nouns = p2_nouns[0:2]
    p1_nouns = p1_nouns[0:2]
    new_text_n = transposeAdjectives(p1_nouns, p2_nouns, new_text_adj)
    print("--------------new text ( after substituting nouns )-------")
    print(new_text_n)

    p2_adverbs = poet2["all_adverbs"]
    p1_adverbs = poet1["all_adverbs"]
    p2_adverbs= p2_adverbs[0:2]
    p1_adverbs = p1_adverbs[0:2]
    brand_new_text = transposeAdjectives(p1_adverbs, p2_adverbs, new_text_n)
    print("--------------new text ( after substituting adverbs )-------")
    print("--------------FINAL SWAPPED POEM----------------------------")

    print(brand_new_text)
    
    return brand_new_text

### **Substituting Sylvia Poems with Swapped Verbs, Adjectives , Noun and Adverbs of Rabindranath Poems and validate the intermediate poems for any similarity or sense.**

In [ ]:
Sylvia_Plath_poems_2 = Sylvia_Plath_poems.copy()
Sylvia_Plath_poems_2.pop('poems_array')
Sylvia_Plath_poems_swapped=[]
for i in Sylvia_Plath_poems:
  print("poems:")
  print(i)
  poem_str = str(Sylvia_Plath_poems[i]);

  Sylvia_Plath_poems_swapped.append(swapIteration(Rabindranth_Tagore_pos, Sylvia_Plath_pos, poem_str))

poems:
poem0
['cross', 'drop']
['red', 'pulled']
tensor([[0.1307]])
tensor([[0.2321]])
tensor([[0.1150]])
tensor([[0.2572]])
--------------new text ( after substituting verbs )-------
the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses
tensor([[0.2145]])
tensor([[0.1147]])
tensor([[0.1798]])
tensor([[0.0950]])
--------------new text ( after substituting adjectives )-------
the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i s

### **Printing the new poems after substituting Sylvia Plath's poems**

In [ ]:
Sylvia_Plath_poems_swapped

['the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses',
 'compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away',
 'stasis in darkness then the substanceless blue pour of tor 

### **Substituting Rabindranath Poems with Swapped Verbs, Adjectives , Noun and Adverbs of Sylvia's  Poems and validate the intermediate poems for any similarity or sense.**

In [ ]:
Rabindranth_Tagore_poems_swapped = []
Rabindranth_Tagore_poems_2 = Rabindranth_Tagore_poems.copy()
Rabindranth_Tagore_poems_2.pop('poems_array')
for i in Rabindranth_Tagore_poems:
  print("poems:")
  print(i)
  # inx = 'poem'+str(i);
  poem_str_2 = str(Rabindranth_Tagore_poems[i]);
  print(poem_str_2)

  Rabindranth_Tagore_poems_swapped.append(swapIteration(Sylvia_Plath_pos, Rabindranth_Tagore_pos, poem_str_2))

poems:
poem0
if people came to know where my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper where my king s palace is it is at the corner of our terrace where the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas there is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is there in the corner of our terrace where the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner where the shadow of the walls mee

### **Printing the new poems after substituting Rabindranath Tagore's poems**

In [ ]:
Rabindranth_Tagore_poems_swapped

['if people came to know wmost my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper wmost my king s palace is it is at the corner of our terrace wmost the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas tmost is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is tmost in the corner of our terrace wmost the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner wmost the shadow of the walls meet together 

### **Build the Swapped Poems in an Array**

In [ ]:
def buildSwappedPoemArray(poet,data):
  poems = []
  data_length = len(data)
  for i in range(data_length):
    poem = data[i]
    #clean_poem = cleanPoemText(poem)
    idx = "poem" + str(i)
    poet[idx] = poem
    poems.append(idx)
  poet['poems_swapped_array'] = poems

In [ ]:
Sylvia_Plath_swapped = {

}

Rabindranth_Tagore_swapped = {

}

buildSwappedPoemArray(Sylvia_Plath_swapped,Sylvia_Plath_poems_swapped)
buildSwappedPoemArray(Rabindranth_Tagore_swapped,Rabindranth_Tagore_poems_swapped)

In [ ]:
Sylvia_Plath_swapped

{'poem0': 'the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses',
 'poem1': 'compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away',
 'poem10': "['poem0', 'poem1', 'poem2', 'po

In [ ]:
Sylvia_Plath_swapped_to_json_file = Sylvia_Plath_swapped.copy()
Sylvia_Plath_swapped_to_json_file.pop('poems_swapped_array')
Sylvia_Plath_swapped_to_json_file.pop('poem10')

Rabindranth_Tagore_swapped_to_json_file = Rabindranth_Tagore_swapped.copy()
Rabindranth_Tagore_swapped_to_json_file.pop('poems_swapped_array')
Rabindranth_Tagore_swapped_to_json_file.pop('poem10')


"['poem0', 'poem1', 'poem2', 'poem3', 'poem4', 'poem5', 'poem6', 'poem7', 'poem8', 'poem9']"

In [ ]:
Sylvia_Plath_swapped_to_json_file

{'poem0': 'the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses',
 'poem1': 'compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away',
 'poem2': 'stasis in darkness then the subs

In [ ]:
Rabindranth_Tagore_swapped_to_json_file

{'poem0': 'if people came to know wmost my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper wmost my king s palace is it is at the corner of our terrace wmost the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas tmost is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is tmost in the corner of our terrace wmost the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner wmost the shadow of the walls meet 

###**Store to Resultant Swapped Poems for both Poets in Text and Json files**

In [ ]:
with open('Sylvia_Plath_swapped_to_json_file_final.json', 'w') as outfile:
    json.dump(Sylvia_Plath_swapped_to_json_file, outfile)

In [ ]:
with open('Rabindranth_Tagore_swapped_to_json_file_final.json', 'w') as outfile:
    json.dump(Rabindranth_Tagore_swapped_to_json_file, outfile)

In [ ]:
with open('Sylvia_Plath_swapped_to_txt_file_final.txt', 'w') as outfile:
    json.dump(Sylvia_Plath_swapped_to_json_file, outfile)

In [ ]:
with open('Rabindranth_Tagore_swapped_to_txt_file_final.txt', 'w') as outfile:
    json.dump(Rabindranth_Tagore_swapped_to_json_file, outfile)

## **Apply Latent Dirichlet Allocation - LDA Topic Modelling**

In [ ]:
Sylvia_Plath_choherence_input = Sylvia_Plath.copy()
Sylvia_Plath_choherence_input.pop('poems_array')
Sylvia_Plath_choherence_input

{'poem0': 'the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses',
 'poem1': 'compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away',
 'poem2': 'stasis in darkness then the subs

In [ ]:
swapped_Sylvia_Plath_choherence_input = Sylvia_Plath_swapped.copy()
swapped_Sylvia_Plath_choherence_input.pop('poems_swapped_array')
swapped_Sylvia_Plath_choherence_input

{'poem0': 'the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses',
 'poem1': 'compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away',
 'poem10': "['poem0', 'poem1', 'poem2', 'po

In [ ]:
Rabindranth_tagore_choherence_input = Rabindranth_Tagore.copy()
Rabindranth_tagore_choherence_input.pop('poems_array')
Rabindranth_tagore_choherence_input 

{'poem0': 'if people came to know where my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper where my king s palace is it is at the corner of our terrace where the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas there is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is there in the corner of our terrace where the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner where the shadow of the walls meet 

In [ ]:
swapped_Rabindranth_tagore_choherence_input  = Rabindranth_Tagore_swapped.copy()
swapped_Rabindranth_tagore_choherence_input.pop('poems_swapped_array')
swapped_Rabindranth_tagore_choherence_input 

{'poem0': 'if people came to know wmost my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper wmost my king s palace is it is at the corner of our terrace wmost the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas tmost is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is tmost in the corner of our terrace wmost the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner wmost the shadow of the walls meet 

## **Run Topic Modelling on combined text**

Here we have build the below models:
1. Bigram Model
2. Trigram Model
3. Function to remove Stopwords, bigrams, trigrams and lemmatization.
4. Create Dictionary.
5. Create Corpus needed for topic modeling.


In [ ]:
def choherence_perplexity_calculator(poem_list):
  def sent_to_words(sentences):
      for sentence in sentences:
          yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

  data_words = list(sent_to_words(poem_list))

  print(data_words[:1])
  # Build the bigram and trigram models
  bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

  # Faster way to get a sentence clubbed as a trigram/bigram
  bigram_mod = gensim.models.phrases.Phraser(bigram)
  trigram_mod = gensim.models.phrases.Phraser(trigram)

  # See trigram example
  print(trigram_mod[bigram_mod[data_words[0]]])
  # Define functions for stopwords, bigrams, trigrams and lemmatization
  def remove_stopwords(texts):
      return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

  def make_bigrams(texts):
      return [bigram_mod[doc] for doc in texts]

  def make_trigrams(texts):
      return [trigram_mod[bigram_mod[doc]] for doc in texts]

  def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
      """https://spacy.io/api/annotation"""
      texts_out = []
      for sent in texts:
          doc = nlp(" ".join(sent)) 
          texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
      return texts_out
  # Remove Stop Words
  #data_words_nostops = remove_stopwords(data_words)

  # Form Bigrams
  data_words_bigrams = make_bigrams(data_words)

  # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
  # python3 -m spacy download en
  nlp = spacy.load('en', disable=['parser', 'ner'])

  # Do lemmatization keeping only noun, adj, vb, adv
  data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN']) #, 'ADJ', 'VERB', 'ADV'

  print(data_lemmatized[:1])
  # Create Dictionary
  id2word = corpora.Dictionary(data_lemmatized)

  # Create Corpus
  texts = data_lemmatized

  # Term Document Frequency
  corpus = [id2word.doc2bow(text) for text in texts]

  # View
  print(corpus[:1])

  lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)
  print(lda_model.print_topics())
  # Visualize the topics
  pyLDAvis.enable_notebook()
  vis = gensimvis.prepare(lda_model, corpus, id2word)
  # vis
  # Compute Perplexity
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  return vis


In [ ]:
poem_count = len(Sylvia_Plath_choherence_input)
Sylvia_Plath_poem_list=[]
for j in Sylvia_Plath_choherence_input:
  poem_index =  str(j)
  print(poem_index)
  print(Sylvia_Plath_choherence_input[poem_index])
  Sylvia_Plath_poem_list.append(Sylvia_Plath_choherence_input[poem_index])
print(Sylvia_Plath_poem_list)

poem0
the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses
poem1
compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away
poem2
stasis in darkness then the substanceless blue pour

### **Original Sylvia Plath Poem's perplexity and coherence score**


In [ ]:
!pip install attrs==19.1.0

In [ ]:
vis = choherence_perplexity_calculator(Sylvia_Plath_poem_list)

[['the', 'womb', 'rattles', 'its', 'pod', 'the', 'moon', 'discharges', 'itself', 'from', 'the', 'tree', 'with', 'nowhere', 'to', 'go', 'my', 'landscape', 'is', 'hand', 'with', 'no', 'lines', 'the', 'roads', 'bunched', 'to', 'knot', 'the', 'knot', 'myself', 'myself', 'the', 'rose', 'you', 'acheive', 'this', 'body', 'this', 'ivory', 'ungodly', 'as', 'child', 'shriek', 'spiderlike', 'spin', 'mirrors', 'loyal', 'to', 'my', 'image', 'uttering', 'nothing', 'but', 'blood', 'taste', 'it', 'dark', 'red', 'and', 'my', 'forest', 'my', 'funeral', 'and', 'this', 'hill', 'and', 'this', 'gleaming', 'with', 'the', 'mouths', 'of', 'corpses']]
['the', 'womb', 'rattles', 'its', 'pod', 'the', 'moon', 'discharges', 'itself', 'from', 'the', 'tree', 'with', 'nowhere', 'to', 'go', 'my', 'landscape', 'is', 'hand', 'with', 'no', 'lines', 'the', 'roads', 'bunched', 'to', 'knot', 'the', 'knot', 'myself', 'myself', 'the', 'rose', 'you', 'acheive', 'this', 'body', 'this', 'ivory', 'ungodly', 'as', 'child', 'shriek'

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['womb', 'pod', 'moon', 'tree', 'landscape', 'hand', 'line', 'road', 'knot', 'rose', 'body', 'ivory', 'child', 'shriek', 'spin', 'mirror', 'image', 'blood', 'taste', 'forest', 'funeral', 'hill', 'gleaming', 'mouth', 'corpse']]
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]]
[(0, '0.049*"daddy" + 0.033*"man" + 0.025*"foot" + 0.025*"year" + 0.025*"heart" + 0.025*"war" + 0.017*"tongue" + 0.017*"town" + 0.017*"root" + 0.017*"pack"'), (1, '0.016*"gull" + 0.016*"cry" + 0.016*"ghost" + 0.016*"tree" + 0.016*"silence" + 0.016*"grass" + 0.008*"year" + 0.008*"eye" + 0.008*"hand" + 0.008*"day"'), (2, '0.017*"blood" + 0.017*"house" + 0.017*"smoke" + 0.017*"pyre" + 0.017*"housewife" + 0.017*"magnet" + 0.017*"injury" + 0.017*"upholstery" + 0.017*"hunter" + 0.017*"move"'), (3, '0.025*"charge" + 0.019*"hand" + 0.019*"ash" + 0.019*"tim

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)



Perplexity:  -6.376773129217327

Coherence Score:  0.7484862244286645


In [ ]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.031246 -0.142473  1       1        22.566342
3     -0.094466  0.101964  2       1        21.356599
1      0.094380  0.113689  3       1        15.969656
0     -0.175432 -0.042929  4       1        14.580947
5      0.070815 -0.071259  5       1        9.905650 
9      0.040695  0.022934  6       1        6.770744 
8      0.028910 -0.003200  7       1        4.342523 
2      0.001189  0.016938  8       1        4.169180 
6      0.000695  0.000479  9       1        0.169180 
7      0.001968  0.003857  10      1        0.169180 , topic_info=        Term      Freq     Total Category  logprob  loglift
133  daddy    4.000000  4.000000  Default  30.0000  30.0000
389  tulip    5.000000  5.000000  Default  29.0000  29.0000
70   sister   3.000000  3.000000  Default  28.0000  28.0000
150  man      3.000000  3.000000  Default  27.0000  27.0000
223  charge   3.000000  3.000000  Default  26.0000  26.0000
..      ...        ...       ...      ...      ...      ...
304  light    0.002547  2.712860  Topic10 -5.9471  -0.5890 
218  breath   0.002546  3.467148  Topic10 -5.9473  -0.8345 
2    child    0.002546  3.672212  Topic10 -5.9475  -0.8922 
322  sight    0.002545  1.945960  Topic10 -5.9477  -0.2574 
58   glitter  0.002544  1.930750  Topic10 -5.9481  -0.2499 

[533 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
267   3      0.847834  absence     
48    1      0.568183  air         
48    2      0.568183  air         
331   1      0.801730  anaesthetist
332   1      0.801384  animal      
...  ..           ...           ...
329   3      0.848085  writhen     
178   1      0.210801  year        
178   2      0.210801  year        
178   3      0.210801  year        
178   4      0.421603  year        

[353 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 1, 6, 10, 9, 3, 7, 8])

If we hover at the bubble then the salient keywords that form the topic will turn red.

As seen from the above graph for topic 1 the salient keywords are:
1. Tulip
2. Water
3. Eye
4. Name
5. Hand
6. Breath
7. Thing etc.

Similarly we can get the salient keywords for other topics also.

Sylvia's poems shows deliberated use of details of her everyday life as the raw material for her art. Due to her case of severe depression, we see most of her works revolve around topics that are resonates her feeling of loneliness,anxiety , confusion and yearn for true love (symbol for Tulip). 

### **Original Rabindranath Tagore Poem's perplexity and coherence score**

In [ ]:
poem_count = len(Rabindranth_tagore_choherence_input)
rabindranath_tagore_list=[]
for j in Rabindranth_tagore_choherence_input:
  poem_index =  str(j)
  print(poem_index)
  print(Rabindranth_tagore_choherence_input[poem_index])
  rabindranath_tagore_list.append(Rabindranth_tagore_choherence_input[poem_index])
print(rabindranath_tagore_list)

poem0
if people came to know where my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper where my king s palace is it is at the corner of our terrace where the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas there is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is there in the corner of our terrace where the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner where the shadow of the walls meet toget

In [ ]:
vis = choherence_perplexity_calculator(rabindranath_tagore_list)

[['if', 'people', 'came', 'to', 'know', 'where', 'my', 'king', 'palace', 'is', 'it', 'would', 'vanish', 'into', 'the', 'air', 'the', 'walls', 'are', 'of', 'white', 'silver', 'and', 'the', 'roof', 'of', 'shining', 'gold', 'the', 'queen', 'lives', 'in', 'palace', 'with', 'seven', 'courtyards', 'and', 'she', 'wears', 'jewel', 'that', 'cost', 'all', 'the', 'wealth', 'of', 'seven', 'kingdoms', 'but', 'let', 'me', 'tell', 'you', 'mother', 'in', 'whisper', 'where', 'my', 'king', 'palace', 'is', 'it', 'is', 'at', 'the', 'corner', 'of', 'our', 'terrace', 'where', 'the', 'pot', 'of', 'the', 'tulsi', 'plant', 'stands', 'the', 'princess', 'lies', 'sleeping', 'on', 'the', 'far', 'away', 'shore', 'of', 'the', 'seven', 'impassable', 'seas', 'there', 'is', 'none', 'in', 'the', 'world', 'who', 'can', 'find', 'her', 'but', 'myself', 'she', 'has', 'bracelets', 'on', 'her', 'arms', 'and', 'pearl', 'drops', 'in', 'her', 'ears', 'her', 'hair', 'sweeps', 'down', 'upon', 'the', 'floor', 'she', 'will', 'wake',

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['if', 'people', 'came', 'to', 'know', 'where', 'my', 'king', 'palace', 'is', 'it', 'would', 'vanish', 'into', 'the', 'air', 'the', 'walls', 'are', 'of', 'white', 'silver', 'and', 'the', 'roof', 'of', 'shining', 'gold', 'the', 'queen', 'lives', 'in', 'palace', 'with', 'seven', 'courtyards', 'and', 'she', 'wears', 'jewel', 'that', 'cost', 'all', 'the', 'wealth', 'of', 'seven', 'kingdoms', 'but', 'let', 'me', 'tell', 'you', 'mother', 'in', 'whisper', 'where', 'my', 'king', 'palace', 'is', 'it', 'is', 'at', 'the', 'corner', 'of', 'our', 'terrace', 'where', 'the', 'pot', 'of', 'the', 'tulsi', 'plant', 'stands', 'the', 'princess', 'lies', 'sleeping', 'on', 'the', 'far', 'away', 'shore', 'of', 'the', 'seven', 'impassable', 'seas', 'there', 'is', 'none', 'in', 'the', 'world', 'who', 'can', 'find', 'her', 'but', 'myself', 'she', 'has', 'bracelets', 'on', 'her', 'arms', 'and', 'pearl', 'drops', 'in', 'her', 'ears', 'her', 'hair', 'sweeps', 'down', 'upon', 'the', 'floor', 'she', 'will', 'wake', 

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)



Perplexity:  -6.45963681895351

Coherence Score:  0.3234594737224857


In [ ]:
vis


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.090409 -0.017511  1       1        78.396441
4     -0.134804 -0.002165  2       1        6.800723 
9      0.117513 -0.096529  3       1        4.578192 
7      0.089012  0.135207  4       1        4.036784 
0      0.036733 -0.064034  5       1        3.633137 
2     -0.008139  0.016304  6       1        1.414243 
1     -0.015285  0.007835  7       1        0.944608 
3      0.001920  0.007140  8       1        0.065290 
5      0.001117  0.006497  9       1        0.065290 
8      0.002342  0.007258  10      1        0.065290 , topic_info=       Term       Freq      Total Category  logprob  loglift
49   love    24.000000  24.000000  Default  30.0000  30.0000
48   light   16.000000  16.000000  Default  29.0000  29.0000
422  sky     19.000000  19.000000  Default  28.0000  28.0000
336  night   21.000000  21.000000  Default  27.0000  27.0000
122  cloud   15.000000  15.000000  Default  26.0000  26.0000
..     ...         ...        ...      ...      ...      ...
37   wealth  0.001721   5.635989   Topic10 -6.3348  -0.7601 
47   lamp    0.001723   7.227758   Topic10 -6.3334  -1.0075 
439  spring  0.001720   6.085565   Topic10 -6.3352  -0.8373 
166  earth   0.001730   13.246138  Topic10 -6.3293  -1.6091 
72   beauty  0.001724   7.996694   Topic10 -6.3327  -1.1079 

[572 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
55    1      0.569902  age    
55    2      0.284951  age    
0     1      0.854280  air    
0     4      0.213570  air    
560   2      1.018741  altar  
...  ..           ...  ...    
39    1      0.872403  world  
39    4      0.124629  world  
39    5      0.124629  world  
518   1      0.541793  writing
518   3      0.541793  writing

[309 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 10, 8, 1, 3, 2, 4, 6, 9])

If we hover at the bubble then the salient keywords that form the topic will turn red.
As seen from the above graph for topic 1 the salient keywords are:
1. Flower
2. Love
3. Life
4. night
5. day
6. sky
7. sun
8. light etc.

Similarly we can get the salient keywords for other topics also.

Being the philospher , Rabindranath Tagore is, his work clearly shows what he stands for his philosophy for life and nature.

In [ ]:
swapped_sylvia_plath_poem_list=[]
for j in swapped_Sylvia_Plath_choherence_input: 
  swapped_poem_index =  str(j)
  print(swapped_poem_index)
  print(swapped_Sylvia_Plath_choherence_input[swapped_poem_index])
  swapped_sylvia_plath_poem_list.append(swapped_Sylvia_Plath_choherence_input[swapped_poem_index])
print(swapped_sylvia_plath_poem_list)

poem0
the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses
poem1
compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away
poem2
stasis in darkness then the substanceless blue pour

### **Swapped Slyvia Plath Poem's perplexity and coherence score**

In [ ]:
vis = choherence_perplexity_calculator(swapped_sylvia_plath_poem_list)

[['the', 'womb', 'rattles', 'its', 'pod', 'the', 'moon', 'discharges', 'itself', 'from', 'the', 'tree', 'with', 'nowhere', 'to', 'go', 'my', 'landscape', 'is', 'hand', 'with', 'no', 'lines', 'the', 'roads', 'bunched', 'to', 'knot', 'the', 'knot', 'myself', 'myself', 'the', 'rose', 'you', 'acheive', 'this', 'body', 'this', 'ivory', 'ungodly', 'as', 'child', 'shriek', 'spiderlike', 'spin', 'mirrors', 'loyal', 'to', 'my', 'image', 'uttering', 'nothing', 'but', 'blood', 'taste', 'it', 'dark', 'red', 'and', 'my', 'forest', 'my', 'funeral', 'and', 'this', 'hill', 'and', 'this', 'gleaming', 'with', 'the', 'mouths', 'of', 'corpses']]
['the', 'womb', 'rattles', 'its', 'pod', 'the', 'moon', 'discharges', 'itself', 'from', 'the', 'tree', 'with', 'nowhere', 'to', 'go', 'my', 'landscape', 'is', 'hand', 'with', 'no', 'lines', 'the', 'roads', 'bunched', 'to', 'knot', 'the', 'knot', 'myself', 'myself', 'the', 'rose', 'you', 'acheive', 'this', 'body', 'this', 'ivory', 'ungodly', 'as', 'child', 'shriek'

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[['womb', 'pod', 'moon', 'tree', 'landscape', 'hand', 'line', 'road', 'knot', 'rose', 'body', 'ivory', 'child', 'shriek', 'spin', 'mirror', 'image', 'blood', 'taste', 'forest', 'funeral', 'hill', 'gleaming', 'mouth', 'corpse']]
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]]
[(0, '0.049*"daddy" + 0.033*"man" + 0.025*"foot" + 0.025*"year" + 0.025*"heart" + 0.025*"war" + 0.017*"tongue" + 0.017*"town" + 0.017*"root" + 0.017*"pack"'), (1, '0.016*"gull" + 0.016*"cry" + 0.016*"ghost" + 0.016*"tree" + 0.016*"silence" + 0.016*"grass" + 0.008*"year" + 0.008*"eye" + 0.008*"day" + 0.008*"hand"'), (2, '0.017*"blood" + 0.017*"house" + 0.017*"smoke" + 0.017*"pyre" + 0.017*"housewife" + 0.017*"magnet" + 0.017*"injury" + 0.017*"upholstery" + 0.017*"hunter" + 0.017*"move"'), (3, '0.025*"charge" + 0.019*"hand" + 0.019*"ash" + 0.019*"tim

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)



Perplexity:  -6.376343984363808

Coherence Score:  0.7484862244286645


In [ ]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.030885 -0.143564  1       1        22.566342
3     -0.096394  0.101601  2       1        21.356599
1      0.095726  0.113895  3       1        15.969656
0     -0.174468 -0.042660  4       1        14.580946
5      0.070656 -0.070983  5       1        9.905650 
9      0.040919  0.023075  6       1        6.770744 
8      0.029545 -0.002454  7       1        4.342523 
2      0.001278  0.016492  8       1        4.169180 
6      0.000650  0.001571  9       1        0.169180 
7      0.001203  0.003028  10      1        0.169180 , topic_info=        Term      Freq     Total Category  logprob  loglift
133  daddy    4.000000  4.000000  Default  30.0000  30.0000
389  tulip    5.000000  5.000000  Default  29.0000  29.0000
70   sister   3.000000  3.000000  Default  28.0000  28.0000
150  man      3.000000  3.000000  Default  27.0000  27.0000
223  charge   3.000000  3.000000  Default  26.0000  26.0000
..      ...        ...       ...      ...      ...      ...
218  breath   0.002504  3.467393  Topic10 -5.9639  -0.8512 
2    child    0.002504  3.673504  Topic10 -5.9640  -0.9090 
322  sight    0.002504  1.946804  Topic10 -5.9641  -0.2741 
58   glitter  0.002503  1.932021  Topic10 -5.9642  -0.2667 
36   minute   0.002502  1.356007  Topic10 -5.9648   0.0868 

[529 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
267   3      0.847572  absence     
48    1      0.568148  air         
48    2      0.568148  air         
331   1      0.801702  anaesthetist
332   1      0.801556  animal      
...  ..           ...           ...
329   3      0.847678  writhen     
178   1      0.210846  year        
178   2      0.210846  year        
178   3      0.210846  year        
178   4      0.421692  year        

[350 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 2, 1, 6, 10, 9, 3, 7, 8])

As seen from the above graph for topic 1 the salient keywords are:
1. tulip
2. water
3. nurse
4. paper
5. eye

Since both poets are very dissimilar in their way of style. the swapped poems still resonated with the original poet.

In [ ]:
swapped_rabindranath_tagore_poem_list=[]
for j in swapped_Rabindranth_tagore_choherence_input:
  swapped_poem_index =  str(j)
  print(swapped_poem_index)
  print(swapped_Rabindranth_tagore_choherence_input[swapped_poem_index])
  swapped_rabindranath_tagore_poem_list.append(swapped_Rabindranth_tagore_choherence_input[swapped_poem_index])
print(swapped_rabindranath_tagore_poem_list)

poem0
if people came to know wmost my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper wmost my king s palace is it is at the corner of our terrace wmost the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas tmost is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is tmost in the corner of our terrace wmost the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner wmost the shadow of the walls meet toget

### **Swapped Rabindranath Tagore Poem's perplexity and coherence score**


In [ ]:
vis = choherence_perplexity_calculator(swapped_rabindranath_tagore_poem_list)

[['if', 'people', 'came', 'to', 'know', 'wmost', 'my', 'king', 'palace', 'is', 'it', 'would', 'vanish', 'into', 'the', 'air', 'the', 'walls', 'are', 'of', 'white', 'silver', 'and', 'the', 'roof', 'of', 'shining', 'gold', 'the', 'queen', 'lives', 'in', 'palace', 'with', 'seven', 'courtyards', 'and', 'she', 'wears', 'jewel', 'that', 'cost', 'all', 'the', 'wealth', 'of', 'seven', 'kingdoms', 'but', 'let', 'me', 'tell', 'you', 'mother', 'in', 'whisper', 'wmost', 'my', 'king', 'palace', 'is', 'it', 'is', 'at', 'the', 'corner', 'of', 'our', 'terrace', 'wmost', 'the', 'pot', 'of', 'the', 'tulsi', 'plant', 'stands', 'the', 'princess', 'lies', 'sleeping', 'on', 'the', 'far', 'away', 'shore', 'of', 'the', 'seven', 'impassable', 'seas', 'tmost', 'is', 'none', 'in', 'the', 'world', 'who', 'can', 'find', 'her', 'but', 'myself', 'she', 'has', 'bracelets', 'on', 'her', 'arms', 'and', 'pearl', 'drops', 'in', 'her', 'ears', 'her', 'hair', 'sweeps', 'down', 'upon', 'the', 'floor', 'she', 'will', 'wake',

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['if', 'people', 'came', 'to', 'know', 'wmost', 'my', 'king', 'palace', 'is', 'it', 'would', 'vanish', 'into', 'the', 'air', 'the', 'walls', 'are', 'of', 'white', 'silver', 'and', 'the', 'roof', 'of', 'shining', 'gold', 'the', 'queen', 'lives', 'in', 'palace', 'with', 'seven', 'courtyards', 'and', 'she', 'wears', 'jewel', 'that', 'cost', 'all', 'the', 'wealth', 'of', 'seven', 'kingdoms', 'but', 'let', 'me', 'tell', 'you', 'mother', 'in', 'whisper', 'wmost', 'my', 'king', 'palace', 'is', 'it', 'is', 'at', 'the', 'corner', 'of', 'our', 'terrace', 'wmost', 'the', 'pot', 'of', 'the', 'tulsi', 'plant', 'stands', 'the', 'princess', 'lies', 'sleeping', 'on', 'the', 'far', 'away', 'shore', 'of', 'the', 'seven', 'impassable', 'seas', 'tmost', 'is', 'none', 'in', 'the', 'world', 'who', 'can', 'find', 'her', 'but', 'myself', 'she', 'has', 'bracelets', 'on', 'her', 'arms', 'and', 'pearl', 'drops', 'in', 'her', 'ears', 'her', 'hair', 'sweeps', 'down', 'upon', 'the', 'floor', 'she', 'will', 'wake', 

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)



Perplexity:  -6.507422669656928

Coherence Score:  0.2981625384741114


In [ ]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.037675 -0.045019  1       1        78.224230
8     -0.176597 -0.001568  2       1        8.754403 
2      0.046380  0.155668  3       1        4.879994 
5      0.057270 -0.099615  4       1        4.143988 
0     -0.023671  0.000130  5       1        3.673241 
6      0.008285 -0.006818  6       1        0.064829 
4      0.012490 -0.001834  7       1        0.064829 
3      0.012241  0.001412  8       1        0.064829 
7      0.012443 -0.001219  9       1        0.064829 
9      0.013484 -0.001136  10      1        0.064829 , topic_info=       Term       Freq      Total Category  logprob  loglift
199  flower  31.000000  31.000000  Default  30.0000  30.0000
49   love    25.000000  25.000000  Default  29.0000  29.0000
450  sun     19.000000  19.000000  Default  28.0000  28.0000
282  life    22.000000  22.000000  Default  27.0000  27.0000
422  sky     20.000000  20.000000  Default  26.0000  26.0000
..   ...           ...        ...      ...      ...      ...
336  night   0.001717   22.542831  Topic10 -6.3369  -2.1411 
480  tree    0.001709   15.678511  Topic10 -6.3419  -1.7830 
166  earth   0.001705   13.248114  Topic10 -6.3440  -1.6166 
214  fruit   0.001702   10.847155  Topic10 -6.3460  -1.4187 
145  day     0.001714   22.136667  Topic10 -6.3391  -2.1251 

[597 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
53    1      0.783171  abandonment
55    1      0.588167  age        
55    2      0.588167  age        
0     1      0.853126  air        
0     3      0.213281  air        
...  ..           ...  ...        
518   2      0.317559  work       
39    1      0.867427  world      
39    3      0.123918  world      
39    5      0.123918  world      
521   2      0.612149  writing    

[327 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 9, 3, 6, 1, 7, 5, 4, 8, 10])

As seen from the above graph for topic 1 the salient keywords are:
1. flower
2. love
3. life
4. night
5. day

Since both poets are very dissimilar in their way of style. the swapped poems still resonated with the original poet.

In [ ]:
 !python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 30.5 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.max_length = 3000000

In [ ]:
def spacy_tokenizer(sentence):
    return [word.lemma_ for word in nlp(sentence) if not (word.like_num or word.is_stop or word.is_punct or word.is_space or len(word)==1)]

In [ ]:
vectorizer_sylvia = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)
vectorizer_rabindranath = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)

data_vectorized_sylvia = vectorizer_sylvia.fit_transform(tqdm(Sylvia_Plath_poem_list))
data_vectorized_rabindranath= vectorizer_rabindranath.fit_transform(tqdm(rabindranath_tagore_list))

100%|██████████| 10/10 [00:01<00:00,  8.03it/s]


### **Combine poems, Summarize using tansformer and Save in file**

In [ ]:
def combinePoems(poems, poet):
    poems_ref = []
    all_poems = []
    c = 0
    for i in poems:
      c += 1
      # print(i)
      all_poems.append(i)
      if c == 10:
        break
    p = {'all_poems':all_poems}
    poems_ref.append(p)
    po = {'poet': poet}
    poems_ref.append(po)
    return poems_ref

In [ ]:
sylvia_combined = swapped_sylvia_plath_poem_list.copy()
sylvia_combined = combinePoems(sylvia_combined, 'Sylvia Plath')
sylvia_combined

[{'all_poems': ['the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses',
   'compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away',
   'stasis in darkness then the substanceles

In [ ]:
rabindranath_combined = swapped_rabindranath_tagore_poem_list.copy()
rabindranath_combined = combinePoems(rabindranath_combined, 'Rabindranath Tagore')
rabindranath_combined

[{'all_poems': ['if people came to know wmost my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper wmost my king s palace is it is at the corner of our terrace wmost the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas tmost is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is tmost in the corner of our terrace wmost the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner wmost the shadow of the walls

In [ ]:
with open('sylvia_plath_swapped_combined_final.json', 'w') as outfile:
    json.dump(sylvia_combined, outfile)
with open('rabindranath_tagore_swapped_combined_final.json', 'w') as outfile:
    json.dump(rabindranath_combined, outfile)

In [ ]:
vectorizer_sylvia = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)
vectorizer_rabindranath = CountVectorizer(tokenizer = spacy_tokenizer, min_df=2)

LDA understands and learns that hidden topic exists but is not yet developed. In other words, LDA aids to extract from the data are also “hidden topics”.

## **Apply Latent Semantic Analysis - LSA Topic Modelling**

###On Sylvia Plath Poems

In [ ]:
# preview of dataset
SP_DF1.head()

,Unnamed: 0,Title,Poem,Poet
0,0,Childless Woman,"The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.",Sylvia Plath
1,1,Aftermath,"Compelled by calamity's magnet They loiter and stare as if the house Burnt-out were theirs, or as if they thought Some scandal might any minute ooze From a smoke-choked closet into light; No deaths, no prodigious injuries Glut these hunters after an old meat, Blood-spoor of the austere tragedies. Mother Medea in a green smock Moves humbly as any housewife through Her ruined apartments, taking stock Of charred shoes, the sodden upholstery: Cheated of the pyre and the rack, The crowd sucks her last tear and turns away.",Sylvia Plath
2,2,Ariel,"Stasis in darkness. Then the substanceless blue Pour of tor and distances. God's lioness, How one we grow, Pivot of heels and knees! —-The furrow Splits and passes, sister to The brown arc Of the neck I cannot catch, Nigger-eye Berries cast dark Hooks —- Black sweet blood mouthfuls, Shadows. Something else Hauls me through air —- Thighs, hair; Flakes from my heels. White Godiva, I unpeel —- Dead hands, dead stringencies. And now I Foam to wheat, a glitter of seas. The child's cry Melts in the wall. And I Am the arrow, The dew that flies, Suicidal, at one with the drive Into the red Eye, the cauldron of morning.",Sylvia Plath
3,3,Barren Woman,"Empty, I echo to the least footfall, Museum without statues, grand with pillars, porticoes, rotundas. In my courtyard a fountain leaps and sinks back into itself, Nun-hearted and blind to the world. Marble lilies Exhale their pallor like scent. I imagine myself with a great public, Mother of a white Nike and several bald-eyed Apollos. Instead, the dead injure me with attentions, and nothing can happen. Blank-faced and mum as a nurse.",Sylvia Plath
4,4,Amnesiac,"No use, no use, now, begging Recognize!There is nothing to do with such a beautiful blank but smooth it.Name, house, car keys,The little toy wife—Erased, sigh, sigh.Four babies and a cocker!Nurses the size of worms and a minute doctorTuck him in.Old happeningsPeel from his skin.Down the drain with all of it!Hugging his pillowLike the red-headed sister he never dared to touch,He dreams of a new one—Barren, the lot are barren!And of another color.How they'll travel, travel, travel, scenerySparking off their brother-sister rearsA comet tail!And money the sperm fluid of it all.One nurse brings inA green drink, one a blue.They rise on either side of him like stars.The two drinks flame and foam.O sister, mother, wife,Sweet Lethe is my life.I am never, never, never coming home!",Sylvia Plath


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):
    
    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(), 
                                remove_stopwords, 
                                strip_punctuation, 
                                strip_short, 
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    
    return text

In [ ]:
# apply function to all reviews 
SP_DF1['Text (Clean)'] = SP_DF1['Poem'].apply(lambda x: preprocess(x))

In [ ]:
# preview of dataset
SP_DF1.head()

,Unnamed: 0,Title,Poem,Poet,Text (Clean)
0,0,Childless Woman,"The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.",Sylvia Plath,"[womb, rattl, pod, moon, discharg, tree, landscap, hand, line, road, bunch, knot, knot, myself, rose, acheive—, bodi, ivori, ungodli, child, shriek, spiderlik, spin, mirror, loyal, imag, utter, blood—, tast, dark, red, forest, funer, hill, gleam, mouth, corps]"
1,1,Aftermath,"Compelled by calamity's magnet They loiter and stare as if the house Burnt-out were theirs, or as if they thought Some scandal might any minute ooze From a smoke-choked closet into light; No deaths, no prodigious injuries Glut these hunters after an old meat, Blood-spoor of the austere tragedies. Mother Medea in a green smock Moves humbly as any housewife through Her ruined apartments, taking stock Of charred shoes, the sodden upholstery: Cheated of the pyre and the rack, The crowd sucks her last tear and turns away.",Sylvia Plath,"[compel, calam, magnet, loiter, stare, hous, burnt, out, their, thought, scandal, minut, ooz, smoke, choke, closet, light, death, prodigi, injuri, glut, hunter, old, meat, blood, spoor, auster, tragedi, mother, medea, green, smock, move, humbl, housewif, ruin, apart, take, stock, char, shoe, sodden, upholsteri, cheat, pyre, rack, crowd, suck, tear, turn, awai]"
2,2,Ariel,"Stasis in darkness. Then the substanceless blue Pour of tor and distances. God's lioness, How one we grow, Pivot of heels and knees! —-The furrow Splits and passes, sister to The brown arc Of the neck I cannot catch, Nigger-eye Berries cast dark Hooks —- Black sweet blood mouthfuls, Shadows. Something else Hauls me through air —- Thighs, hair; Flakes from my heels. White Godiva, I unpeel —- Dead hands, dead stringencies. And now I Foam to wheat, a glitter of seas. The child's cry Melts in the wall. And I Am the arrow, The dew that flies, Suicidal, at one with the drive Into the red Eye, the cauldron of morning.",Sylvia Plath,"[stasi, dark, substanceless, blue, pour, tor, distanc, god, lioness, grow, pivot, heel, knee, the, furrow, split, pass, sister, brown, arc, neck, catch, nigger, ey, berri, cast, dark, hook, black, sweet, blood, mouth, shadow, haul, air, thigh, hair, flake, heel, white, godiva, unpeel, dead, hand, dead, stringenc, foam, wheat, glitter, sea, child, melt, wall, arrow, dew, fli, suicid, drive, red, ey, cauldron, morn]"
3,3,Barren Woman,"Empty, I echo to the least footfall, Museum without statues, grand with pillars, porticoes, rotundas. In my courtyard a fountain leaps and sinks back into itself, Nun-hearted and blind to the world. Marble lilies Exhale their pallor like scent. I imagine myself with a great public, Mother of a white Nike and several bald-eyed Apollos. Instead, the dead injure me with attentions, and nothing can happen. Blank-faced and mum as a nurse.",Sylvia Plath,"[empti, echo, footfal, museum, statu, grand, pillar, portico, rotunda, courtyard, fountain, leap, sink, itself, nun, heart, blind, world, marbl, lili, exhal, pallor, like, scent, imagin, great, public, mother, white, nike, bald, ei, apollo, instead, dead, injur, attent, happen, blank, face, mum, nurs]"
4,4,Amnesiac,"No use, no use, now, begging Recognize!There is nothing to do with such a beautiful blank but smooth it.Name, house, car keys,The little toy wife—Erased, sigh, sigh.Four babies and a cocker!Nurses the size of worms and a minute doctorTuck him in.Old happeningsPeel from his skin.Down the drain with all of it!Hugging his pillowLike the red-headed sister he never dared to touch,He dreams of a new one—Barren, the lot are barren!And of another color.How they

We can convert these processed reviews into a document-term matrix with the bag of words model.

In [ ]:
# create a dictionary with the corpus
corpus = SP_DF1['Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

Next, we have to implement the truncated singular value decomposition on this matrix. In the Gensim library, we can use the LSI model to build a model that performs SVD on the given matrix

**Before we can create the lower dimensional matrices, we need to determine the number of topics that should be extracted from these reviews.**

One approach towards finding the best number of topics is using the coherence score metric. The coherence score essentially shows how similar the words from each topic are in terms of semantic value, with a higher score corresponding to higher similarity.

Again, we can obtain the coherence score with the Gensim module. Let’s see how the coherence score is for the range of 2 to 10 topics.

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=SP_DF1['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.30983730553789324
Coherence score with 3 clusters: 0.42557388322515594
Coherence score with 4 clusters: 0.45648467753906485
Coherence score with 5 clusters: 0.5974402514089117
Coherence score with 6 clusters: 0.6250113468632194
Coherence score with 7 clusters: 0.48156740959804417
Coherence score with 8 clusters: 0.6787436386272164
Coherence score with 9 clusters: 0.629266198915518
Coherence score with 10 clusters: 0.6349372098682926


**The coherence score is highest with 9 topics, so that is the number of topics we will extract when performing SVD.**

In [ ]:
# perform SVD on the bag of words with the LsiModel to extract 8 topics
lsi = LsiModel(bow, num_topics=9, id2word=dictionary)

**We are able to obtain 2 topics from the document-term matrix. As a result, we can see which words have the strongest association with each topic and infer what these topics represent.**

Let’s see the 5 words that each topic has the strongest association to.

In [ ]:
# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.393*"like" + 0.365*"you" + 0.187*"black" + 0.156*"tulip" + 0.154*"daddi".
Words in 1: 0.381*"you" + -0.221*"tulip" + 0.193*"daddi" + 0.161*"man" + 0.155*"jew".
Words in 2: -0.342*"the" + -0.163*"blue" + 0.148*"tulip" + -0.117*"like" + 0.108*"white".
Words in 3: 0.238*"charg" + 0.238*"herr" + 0.179*"time" + 0.179*"ash" + -0.148*"the".
Words in 4: 0.279*"travel" + 0.270*"sister" + 0.186*"sigh" + 0.186*"thei" + 0.186*"never".
Words in 5: -0.197*"beat" + -0.191*"sun" + -0.177*"water" + -0.118*"dead" + -0.110*"split".
Words in 6: -0.253*"dark" + -0.233*"heel" + -0.217*"dead" + -0.210*"ey" + 0.136*"like".
Words in 7: 0.140*"auster" + 0.140*"move" + 0.140*"char" + 0.140*"calam" + 0.140*"burnt".
Words in 8: -0.151*"dead" + -0.150*"portico" + -0.150*"nike" + -0.150*"pallor" + -0.150*"mum".


Based on the given words, 
<ul> 
<li>Topic 0 may represent poem that address the color / appearance of sky or view.
<li>Topic 1 may represent poems picturizing the various colors of the evening.

<b><i>Most of the words in the topics are overlapping with other topics. But somehow, we can assume that poet is trying to convey her deeper feeling by using metaphors related to colors. It also mentions somethings similar to dark, sadness, death, heart, 'hanging on', evenings, waits. <i><b>

We know that each poem usually have multiple topics. 

However, some topics have a stronger association with the documents than others. 

So, we can determine which topic a poem belongs to by finding the one that registers the highest value by magnitude

In [ ]:
# find the scores given between the poem and each topic
corpus_lsi = lsi[bow]
score1 = []
score2 = []
score3 = []
score4 = []
score5 = []
score6 = []
score7 = []
score8 = []

for doc in corpus_lsi:
    score1.append(round(doc[0][1],2))
    score2.append(round(doc[1][1],2))
    score3.append(round(doc[2][1],2))
    score4.append(round(doc[3][1],2))
    score5.append(round(doc[4][1],2))
    score6.append(round(doc[5][1],2))
    score7.append(round(doc[6][1],2))
    score8.append(round(doc[7][1],2))

# create data frame that shows scores assigned for both topics for each review
df_topic = pd.DataFrame()
df_topic['Text'] = SP_DF['Poem']
df_topic['Topic 0 score'] = score1
df_topic['Topic 1 score'] = score2
df_topic['Topic 2 score'] = score3
df_topic['Topic 3 score'] = score4
df_topic['Topic 4 score'] = score5
df_topic['Topic 5 score'] = score6
df_topic['Topic 6 score'] = score7
df_topic['Topic 7 score'] = score8

df_topic['Topic']= df_topic[['Topic 0 score', 'Topic 1 score','Topic 2 score', 'Topic 3 score','Topic 4 score', 'Topic 5 score','Topic 6 score', 'Topic 7 score']].apply(lambda x: x.argmax(), axis=1)
df_topic.head(1)


,Text,Topic 0 score,Topic 1 score,Topic 2 score,Topic 3 score,Topic 4 score,Topic 5 score,Topic 6 score,Topic 7 score,Topic
0,"The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.",0.52,-0.61,-0.16,-0.12,-0.07,-0.14,-1.28,-0.1,0


In [ ]:
df_topic

,Text,Topic 0 score,Topic 1 score,Topic 2 score,Topic 3 score,Topic 4 score,Topic 5 score,Topic 6 score,Topic 7 score,Topic
0,"The womb Rattles its pod, the moon Discharges itself from the tree with nowhere to go. My landscape is a hand with no lines, The roads bunched to a knot, The knot myself, Myself the rose you acheive—- This body, This ivory Ungodly as a child's shriek. Spiderlike, I spin mirrors, Loyal to my image, Uttering nothing but blood—- Taste it, dark red! And my forest My funeral, And this hill and this Gleaming with the mouths of corpses.",0.52,-0.61,-0.16,-0.12,-0.07,-0.14,-1.28,-0.10,0
1,"Compelled by calamity's magnet They loiter and stare as if the house Burnt-out were theirs, or as if they thought Some scandal might any minute ooze From a smoke-choked closet into light; No deaths, no prodigious injuries Glut these hunters after an old meat, Blood-spoor of the austere tragedies. Mother Medea in a green smock Moves humbly as any housewife through Her ruined apartments, taking stock Of charred shoes, the sodden upholstery: Cheated of the pyre and the rack, The crowd sucks her last tear and turns away.",0.44,-0.11,-0.26,0.02,0.69,-0.09,0.25,7.08,7
2,"Stasis in darkness. Then the substanceless blue Pour of tor and distances. God's lioness, How one we grow, Pivot of heels and knees! —-The furrow Splits and passes, sister to The brown arc Of the neck I cannot catch, Nigger-eye Berries cast dark Hooks —- Black sweet blood mouthfuls, Shadows. Something else Hauls me through air —- Thighs, hair; Flakes from my heels. White Godiva, I unpeel —- Dead hands, dead stringencies. And now I Foam to wheat, a glitter of seas. The child's cry Melts in the wall. And I Am the arrow, The dew that flies, Suicidal, at one with the drive Into the red Eye, the cauldron of morning.",1.79,-1.13,-0.94,-0.24,0.91,-1.09,-7.85,0.17,0
3,"Empty, I echo to the least footfall, Museum without statues, grand with pillars, porticoes, rotundas. In my courtyard a fountain leaps and sinks back into itself, Nun-hearted and blind to the world. Marble lilies Exhale their pallor like scent. I imagine myself with a great public, Mother of a white Nike and several bald-eyed Apollos. Instead, the dead injure me with attentions, and nothing can happen. Blank-faced and mum as a nurse.",1.13,-0.55,-0.06,0.05,0.43,-0.34,0.05,0.19,0
4,"No use, no use, now, begging Recognize!There is nothing to do with such a beautiful blank but smooth it.Name, house, car keys,The little toy wife—Erased, sigh, sigh.Four babies and a cocker!Nurses the size of worms and a minute doctorTuck him in.Old happeningsPeel from his skin.Down the drain with all of it!Hugging his pillowLike the red-headed sister he never dared to touch,He dreams of a new one—Barren, the lot are barren!And of another color.How they'll travel, travel, travel, scenerySparking off their brother-sister rearsA comet tail!And money the sperm fluid of it all.One nurse brings inA green drink, one a blue.They rise on either side of him like stars.The two drinks flame and foam.O sister, mother, wife,Sweet Lethe is my life.I am never, never, never coming home!",1.81,-1.09,-2.36,-0.51,10.10,1.40,0.67,-0.50,4
5,"You do not do, you do not do Any more, black shoe In which I have lived like a foot For thirty years, poor and white, Barely daring to breathe or Achoo. Daddy, I have had to kill you. You died before I had time—- Marble-heavy, a bag full of God, Ghastly statue with one gray toe Big as a Frisco seal And a head in the freakish Atlantic Where it pours bean green over blue In the waters off the beautiful Nauset. I used to pray to recover you. Ach, du. In the German tongue, in the Polish town Scraped flat by the roller Of wars, wars, wars. But the name of the town is common. My Polack friend Says there are a dozen or two. So I never could tell where you Put your foot, your root, I never could talk to you. The tongue stuck in my jaw. It stuck in a barb wire snare. Ich, ich, ich, ich, I could hardly speak. I thought eve

###Similarly apply same steps on Rabindranath Tagore's Poems

In [ ]:
# preview of dataset
RT_DF1.head()

Unnamed: 0  ...                 Poet
0  0           ...  Rabindranath Tagore
1  1           ...  Rabindranath Tagore
2  2           ...  Rabindranath Tagore
3  3           ...  Rabindranath Tagore
4  4           ...  Rabindranath Tagore

[5 rows x 4 columns]

In [ ]:
# apply function to all reviews 
RT_DF1['Text (Clean)'] = RT_DF1['Poem'].apply(lambda x: preprocess(x))

In [ ]:
# preview of dataset
RT_DF1.head()

Unnamed: 0  ...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                Text (Clean)
0  0           ...  [peopl, came, know, king, palac, vanish, air, wall, white, silver, roof, shine, gold, queen, live, palac, seven, courtyard, wear, jewel, cost, wealth, seven, kingdom, let, tell, you, mother, whisper, king, palac, corner, terrac, pot, tulsi, plant, stand, princess, li, sleep, far, awai, shore, seven, impass, sea, world, myself, bracelet, arm, pearl, drop, ear, hair, sweep, floor, wake, touch, magic, wand, jewel, fall, lip, smile, let, whisper, ear, mother, corner, terrac, pot, tulsi, plant, stand, time, river, bath, step, terrac, roof, sit, corner, shadow, wall, meet, togeth, puss, allow, come, know, barber, stori, live, let, whisper, mother, ear, barber, stori, live, ...]                                                  
1  1           ...  [fool, try, carri, thyself, thy, shoulder, beggar, come, beg, thy, door, leav, thy, burden, hand, bear, all, look, regret, thy, desir, put, light, lamp, touch, breath, unholy—, take, thy, gift, unclean, hand, accept, offer, sacr, love]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
2  2           ...  [fanci, firefli, speck, live, light, twinkl, dark, voic, waysid, pansi, attract, careless, glanc, murmur, desultori, line, drowsi, dark, cave, mind, dream, build, nest, fragment, drop, dai, caravan, spring, scatter, petal, flower, fruit, futur, moment, whim, joi, freed, bond, earth, slumber, rush, numberless, leav, danc, air, dai, word, slight, lightli, danc, time, wave, work, havi, import, gone, down, mind, underground, moth, grow, filmi, wing, farewel, flight, sunset, sky, butterfli, count, month, moment, time, enough, thought, like, spark, ride, wing, surpris, carri, singl, laughter, tree, gaze, love, beauti, shadow, grasp, let, love, like, sunlight, surround, illumin, freedom, dai, colour, vbubbl, float, surfac, ...]
3  3           ...  [father, write, lot, book, write, don, understand, read, even, meant, nice, store, mother, tell, can, father, write, like, that, wonder, hear, mother, stori, giant, fairi, princess, forgotten, all, get, late, bath, time, wait, dish, warm, him, goe, write, forget, father, plai, make, book, plai, father, room, come, what, naughti, child, slightest, nois, sai, don, father, work, what, fun, write, write, father, pen, pencil, write, book, doe, why, cross, then, mother, word, father, write, father, wast, heap, paper, mother, don, mind, all, sheet, boat, with, sai, child, troublesom, ar, think, father, spoil, sheet, sheet, paper, black, mark, side, thi, poem, the, ...]                                                            
4  4           ...  [wonder, know, speech, voic, movement, be, skill, line, melodi, song, joi, sorrow, thought, chain, contain, tear, laughter, work, plai, thought, self, come, end, death, flood, joi, feel, sight, touch, belov, self, shore, shine, 

In [ ]:
from gensim import corpora

# create a dictionary with the corpus
corpus = RT_DF1['Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=RT_DF1['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.29378295272470145
Coherence score with 3 clusters: 0.3797031912340136
Coherence score with 4 clusters: 0.5135231053159749
Coherence score with 5 clusters: 0.4035743611253024
Coherence score with 6 clusters: 0.43712346693240683
Coherence score with 7 clusters: 0.5533320295149159
Coherence score with 8 clusters: 0.37483382421661027
Coherence score with 9 clusters: 0.4044275965932924
Coherence score with 10 clusters: 0.46119556259870664


In [ ]:
# perform SVD on the bag of words with the LsiModel to extract 5 topics
lsi = LsiModel(bow, num_topics=5, id2word=dictionary)

In [ ]:
# find the 5 words with the srongest association to the derived topics
for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.303*"love" + 0.303*"flower" + 0.193*"life" + 0.185*"dai" + 0.185*"light".
Words in 1: -0.415*"father" + -0.363*"write" + -0.349*"mother" + -0.165*"plai" + -0.163*"book".
Words in 2: -0.332*"father" + -0.302*"write" + 0.217*"know" + 0.199*"terrac" + 0.199*"corner".
Words in 3: 0.187*"corner" + 0.187*"terrac" + -0.177*"shall" + -0.167*"wave" + -0.165*"cloud".
Words in 4: -0.564*"thy" + -0.416*"forget" + -0.208*"keen" + -0.206*"flute" + -0.152*"far".


**Inference:**

Based on the given words, topic 0 may represent poem that address the love and beauty of live, while topic 1 may represent poems picturizing the various relationships.

topic 2 may represent poem addressing family and its surrounding like home.

topic 3 may represent poem addressing deep thought state o mind and topic 4 may represent the feel of the nature during deep thoughts.

In [ ]:
# find the scores given between the review and each topic
corpus_lsi = lsi[bow]
score1 = []
score2 = []
for doc in corpus_lsi:
    score1.append(round(doc[0][1],2))
    score2.append(round(doc[1][1],2))

# create data frame that shows scores assigned for both topics for each review
df_topic = pd.DataFrame()
df_topic['Text'] = RT_DF['Poem']
df_topic['Topic 0 score'] = score1
df_topic['Topic 1 score'] = score2
df_topic['Topic']= df_topic[['Topic 0 score', 'Topic 1 score']].apply(lambda x: x.argmax(), axis=1)
df_topic.head(1)


,Text,Topic 0 score,Topic 1 score,Topic
0,"If people came to know where my king's palace is, it would vanish into the air. The walls are of white silver and the roof of shining gold. The queen lives in a palace with seven courtyards, and she wears a jewel that cost all the wealth of seven kingdoms. But let me tell you, mother, in a whisper, where my king's palace is. It is at the corner of our terrace where the pot of the tulsi plant stands. The princess lies sleeping on the far-away shore of the seven impassable seas. There is none in the world who can find her but myself. She has bracelets on her arms and pearl drops in her ears; her hair sweeps down upon the floor. She will wake when I touch her with my magic wand and jewels will fall from her lips when she smiles. But let me whisper in your ear, mother; she is there in the corner of our terrace where the pot of the tulsi plant stands. When it is time for you to go to the river for your bath, step up to that terrace on the roof. I sit in the corner where the shadow of the walls meet together. Only puss is allowed to come with me, for she know where the barber in the story lives. But let me whisper, mother, in your ear where the barber in the story lives. It is at the corner of the terrace where the pot of the tulsi plant stands.",1.92,-5.56,0


## **Conclusion**

Based on the frequency count of words, below are the top 3 topic names for both the poets:
1. Sylvia Plath:

  1. Black 

  2. provide

  3. purpose

2. Rabindranath Tagore:

  1. Love

  2. Flower

  3. thy

Based on the LDA weights, below are the top 3 topic names for both the poets:
1. Sylvia Plath
    1. Man
    2. Eye
    3. Half

2. Rabindranath Tagore:

    1. Flower
    2. Love
    3. life 


Eventhough LSA is quick and efficient to use, we notice that it does have a few primary drawbacks:

First, the lack of interpretable embeddings i.e we don’t know what the topics are, and the components may be arbitrarily (positive/negative) and
need for really large set of documents and vocabulary to get accurate results
less efficient representation.

## **Summarize the poems by applying transformers with the  t5 model, task = summarization**

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")
def getSummaryOfPoem(poet):
  def removeQuotes(mydata):
    new_data = mydata.strip('“”')
    return new_data
  count = 0
  for key in poet['poems_array']:
    count += 1;
    text = poet[key] 
    text = removeQuotes(text)
    summary_text = summarizer(text, max_length=1000, min_length=5, do_sample=False)[0]['summary_text']
    poet["summary_"+key] = summary_text
  
  return poet

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
Sylvia_Plath_sum = Sylvia_Plath.copy()
Rabindranth_Tagore_sum = Rabindranth_Tagore.copy()

data_Sylvia_Plath = getSummaryOfPoem(Sylvia_Plath_sum)


Your max_length is set to 1000, but you input_length is only 113. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=56)
Your max_length is set to 1000, but you input_length is only 138. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)
Your max_length is set to 1000, but you input_length is only 169. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=84)
Your max_length is set to 1000, but you input_length is only 114. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Your max_length is set to 1000, but you input_length is only 193. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=96)
Token indices sequence length is longer than the specified maximum sequence length for this model (703 > 512). Running this sequence through the model will result in indexing errors
Your max_length is set to 1000, but you inp

In [ ]:
Sylvia_Plath_sum

{'poem0': 'the womb rattles its pod the moon discharges itself from the tree with nowhere to go my landscape is a hand with no lines the roads bunched to a knot the knot myself myself the rose you acheive— this body this ivory ungodly as a child s shriek spiderlike i spin mirrors loyal to my image uttering nothing but blood— taste it dark red and my forest my funeral and this hill and this gleaming with the mouths of corpses',
 'poem1': 'compelled by calamity s magnet they loiter and stare as if the house burnt out were theirs or as if they thought some scandal might any minute ooze from a smoke choked closet into light no deaths no prodigious injuries glut these hunters after an old meat blood spoor of the austere tragedies mother medea in a green smock moves humbly as any housewife through her ruined apartments taking stock of charred shoes the sodden upholstery cheated of the pyre and the rack the crowd sucks her last tear and turns away',
 'poem2': 'stasis in darkness then the subs

In [ ]:
data_Rabindranth_Tagore = getSummaryOfPoem(Rabindranth_Tagore_sum)

Your max_length is set to 1000, but you input_length is only 297. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=148)
Your max_length is set to 1000, but you input_length is only 97. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)
Your max_length is set to 1000, but you input_length is only 316. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=158)
Your max_length is set to 1000, but you input_length is only 238. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=119)
Your max_length is set to 1000, but you input_length is only 114. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=57)
Your max_length is set to 1000, but you input_length is only 179. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=89)
Your max_length is set to 1000, but you input_length is only 147. Yo

In [ ]:
Rabindranth_Tagore_sum

{'poem0': 'if people came to know where my king s palace is it would vanish into the air the walls are of white silver and the roof of shining gold the queen lives in a palace with seven courtyards and she wears a jewel that cost all the wealth of seven kingdoms but let me tell you mother in a whisper where my king s palace is it is at the corner of our terrace where the pot of the tulsi plant stands the princess lies sleeping on the far away shore of the seven impassable seas there is none in the world who can find her but myself she has bracelets on her arms and pearl drops in her ears her hair sweeps down upon the floor she will wake when i touch her with my magic wand and jewels will fall from her lips when she smiles but let me whisper in your ear mother she is there in the corner of our terrace where the pot of the tulsi plant stands when it is time for you to go to the river for your bath step up to that terrace on the roof i sit in the corner where the shadow of the walls meet 

### **Store the summarized JSON Data to File**

In [ ]:
with open('Sylvia_Plath_summary_final.json', 'w') as outfile:
    json.dump(Sylvia_Plath_sum, outfile)

In [ ]:
with open('Rabindranth_Tagore_summary_final.json', 'w') as outfile:
    json.dump(Rabindranth_Tagore_sum, outfile)

## **Scraping the History / News of the Poets:**




###**POET1 - Rabindranath Tagore**

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.request

import re

CLEANR = re.compile('<.*?>')

def cleanhtmml(rawhtml):
  cleantext = re.sub(CLEANR, '', rawhtml)
  return cleantext

url ="https://indianexpress.com/about/rabindranath-tagore/"

In [ ]:
r1= requests.get(url)
coverpage = r1.content

In [ ]:
soup1 = BeautifulSoup(coverpage,'html5lib')
coverpage_news = soup1.find_all('h3')

In [ ]:
coverpage_news

[<h3><a href="https://indianexpress.com/article/express-sunday-eye/whats-the-writing-on-the-wall-at-the-rashtrapati-bhavan-7849203/" title="What's the writing on the wall at the Rashtrapati Bhavan?">What's the writing on the wall at the Rashtrapati Bhavan?</a></h3>,
 <h3><a href="https://indianexpress.com/article/books-and-literature/history-sriniketan-rabindranath-tagore-book-rural-reconstruction-7780075/" title="'History of Sriniketan': Book throws light on Rabindranath Tagore's work for rural reconstruction">'History of Sriniketan': Book throws light on Rabindranath Tagore's work for rural reconstruction</a></h3>,
 <h3><a href="https://indianexpress.com/article/books-and-literature/rabindranath-tagore-literary-prize-to-be-merged-for-year-2021-2022-organisers-7662033/" title="Rabindranath Tagore Literary Prize to be merged for year 2021, 2022: Organisers">Rabindranath Tagore Literary Prize to be merged for year 2021, 2022: Organisers</a></h3>,
 <h3><a href="https://indianexpress.com/

In [ ]:
list_titles = []
n=len(coverpage_news)
print(n)
for x in range(n): 
    print(str.strip(coverpage_news[x].text))
    list_titles.append(str.strip(coverpage_news[x].text))

18
What's the writing on the wall at the Rashtrapati Bhavan?
'History of Sriniketan': Book throws light on Rabindranath Tagore's work for rural reconstruction
Rabindranath Tagore Literary Prize to be merged for year 2021, 2022: Organisers
Union minister comments on Rabindranath Tagore’s complexion, kicks up row
Is the mind without fear?
Rabindranath Tagore death anniversary: Motivational quotes by the polymath
Vidya Balan drapes 'Ekla cholo re' sari: Know more about this Tagore song
Rabindranath Tagore's birth anniversary: Some of his writings you can revisit
In battle for Bolpur, future of Tagore’s legacy & Visva-Bharati in focus
Modern Indian Art sale: Never before auctioned works by Indian modernists to go under the hammer
After Modi's Visva-Bharati address, Mamata's show of strength at Santiniketan
Kolkata: Basanta Utsav celebrations ahead of Holi
Rabindranath Tagore Jayanti: Pearls of wisdom by the 'Bard' on his birth anniversary
Rabindranath Tagore's 77th death anniversary: Rare 

In [ ]:
list_titles.pop(17)


'Best of Express'

In [ ]:
list_titles

["What's the writing on the wall at the Rashtrapati Bhavan?",
 "'History of Sriniketan': Book throws light on Rabindranath Tagore's work for rural reconstruction",
 'Rabindranath Tagore Literary Prize to be merged for year 2021, 2022: Organisers',
 'Union minister comments on Rabindranath Tagore’s complexion, kicks up row',
 'Is the mind without fear?',
 'Rabindranath Tagore death anniversary: Motivational quotes by the polymath',
 "Vidya Balan drapes 'Ekla cholo re' sari: Know more about this Tagore song",
 "Rabindranath Tagore's birth anniversary: Some of his writings you can revisit",
 'In battle for Bolpur, future of Tagore’s legacy & Visva-Bharati in focus',
 'Modern Indian Art sale: Never before auctioned works by Indian modernists to go under the hammer',
 "After Modi's Visva-Bharati address, Mamata's show of strength at Santiniketan",
 'Kolkata: Basanta Utsav celebrations ahead of Holi',
 "Rabindranath Tagore Jayanti: Pearls of wisdom by the 'Bard' on his birth anniversary",
 "

###Scraping Articles for Rabindranath Tagore - Poet 1

In [ ]:
url = "https://indianexpress.com/article/opinion/is-the-mind-without-fear-tagore-7445907/"

header = {'User-Agent':'Mozilla/5.0'}
requests = urllib.request.Request(url,headers=header)
requestread = urllib.request.urlopen(requests).read()
soup = bs.BeautifulSoup(requestread,'html.parser')

all_paragraphs = soup.findAll('p')
all_paragraphs.pop(0)

paragraphs = all_paragraphs[0:9]
paragraphs

news_articleofRabindranth = cleanhtmml(str(paragraphs))
news_articleofRabindranth 

'[With celebrations about to begin to mark 75 years of political freedom from the colonial yoke, it is time to revisit the makers of India’s intellectual life in modern times. It’s also time to rekindle the debate on the links between political and intellectual freedoms. Unlike European countries, from where we have borrowed the concept of a nation, India’s strength lies in its diversity of different cultures, castes, religions, languages. In Australia and America, colonisers forcefully brought in oneness by almost totally exterminating the native population. In contrast, India has a unique history of tolerance for whoever wished to make it their home. Therefore, aggressive nationalism goes against the idea of India – it is highly impractical as well., One prominent public intellectual who had — and still has — enormous influence on the Indian way of thinking is Rabindranath Tagore. In the run-up to Independence Day, it would be appropriate to revisit his idea of a free nation, present

In [ ]:
summary_textofRabindranthNews = summarizer(news_articleofRabindranth, max_length=512, min_length=5, do_sample=False)[0]['summary_text']
print(summary_textofRabindranthNews)

with celebrations about to begin to mark 75 years of political freedom from the colonial yoke, it is time to revisit the makers of India's intellectual life in modern times . india's strength lies in its diversity of different cultures, castes, religions, languages .


###**POET2 - Sylvia Plath**

 ### Scraping Articles for Sylvia Plath - Poet 1

In [ ]:
url ="https://www.newindianexpress.com/cities/bengaluru/2014/oct/28/Remembering-Sylvia-676011.html"


In [ ]:
header = {'User-Agent':'Mozilla/5.0'}
requests = urllib.request.Request(url,headers=header)
requestread = urllib.request.urlopen(requests).read()
soup = bs.BeautifulSoup(requestread,'html.parser')

all_paragraphs = soup.findAll('p')
all_paragraphs.pop(0)

paragraphs = all_paragraphs[0:9]
paragraphs

news_articleofSylvia = cleanhtmml(str(paragraphs))
news_articleofSylvia

"[BANGALORE : October 27th is the 82nd birth anniversary of Sylvia Plath, one of the most important female poets and writers of the 20th century. Her life has been variously celebrated and appropriated by feminist academics, ordinary American housewives and successive generations of students and aspiring poets. Her poetry and fiction drawn from her personal life and experiences, left an indelible impression on the history of modern English literature. , Most discussions and debates around Plath's work touch upon her history and struggle with depression as well as her suicide at the young age of 30. It is this fact combined with Sylvia Plath's turbulent marriage to poet Ted Hughes, her own almost therapeutic examination of her mental state through her writing, her very public attempts to exorcise her personal demons and literary as well as real exploration of the ideas of mortality and suicide (Plath tried to kill herself on numerous occasions), that led to her posthumous reimagining as

In [ ]:
summary_textofSylviaNews = summarizer(news_articleofSylvia, max_length=512, min_length=5, do_sample=False)[0]['summary_text']
print(summary_textofSylviaNews)

'a feminist martyr' and all-round 'Queen of Sorrows' who died at her creative zenith . her innate poetic talent, her elemental imagery and her rich inner life made her a literary icon for all time .


##Check Cosine Similarity of Summaries between Poems and Summary of News Articles

In [ ]:
for poem in Sylvia_Plath_sum['poems_array']:  
  print(poem + ' of Sylvia Plath has cosine similarity with the news data: ' + str(getSemanticSimilarity(summary_textofSylviaNews, Sylvia_Plath_sum[poem])))

tensor([[0.3605]])
poem0 of Sylvia Plath has cosine similarity with the news data: 0.36048075556755066
tensor([[0.2554]])
poem1 of Sylvia Plath has cosine similarity with the news data: 0.25535616278648376
tensor([[0.3597]])
poem2 of Sylvia Plath has cosine similarity with the news data: 0.35965660214424133
tensor([[0.3929]])
poem3 of Sylvia Plath has cosine similarity with the news data: 0.39285147190093994
tensor([[0.2731]])
poem4 of Sylvia Plath has cosine similarity with the news data: 0.27314329147338867
tensor([[0.3059]])
poem5 of Sylvia Plath has cosine similarity with the news data: 0.30593666434288025
tensor([[0.1113]])
poem6 of Sylvia Plath has cosine similarity with the news data: 0.11127905547618866
tensor([[0.4021]])
poem7 of Sylvia Plath has cosine similarity with the news data: 0.4021340012550354
tensor([[0.1897]])
poem8 of Sylvia Plath has cosine similarity with the news data: 0.1896592229604721
tensor([[0.1888]])
poem9 of Sylvia Plath has cosine similarity with the new

The above results shows that poem 8 - Lady Lazarus, 4 - Barren Woman and 1 - Childless Woman resonates more with the topics found in the artice scraped from the news.

In [ ]:
for poem in Rabindranth_Tagore_sum['poems_array']:
  print(poem + ' of Rabindranth Tagore has cosine similarity with the news data: ' + str(getSemanticSimilarity(summary_textofSylviaNews, Rabindranth_Tagore_sum[poem])))

tensor([[0.4137]])
poem0 of Rabindranth Tagore has cosine similarity with the news data: 0.41371357440948486
tensor([[0.3333]])
poem1 of Rabindranth Tagore has cosine similarity with the news data: 0.3332754373550415
tensor([[0.3584]])
poem2 of Rabindranth Tagore has cosine similarity with the news data: 0.35835832357406616
tensor([[0.2476]])
poem3 of Rabindranth Tagore has cosine similarity with the news data: 0.2475840002298355
tensor([[0.3127]])
poem4 of Rabindranth Tagore has cosine similarity with the news data: 0.31268054246902466
tensor([[0.3932]])
poem5 of Rabindranth Tagore has cosine similarity with the news data: 0.3931979537010193
tensor([[0.2423]])
poem6 of Rabindranth Tagore has cosine similarity with the news data: 0.2423085719347
tensor([[0.2993]])
poem7 of Rabindranth Tagore has cosine similarity with the news data: 0.2993481159210205
tensor([[0.3030]])
poem8 of Rabindranth Tagore has cosine similarity with the news data: 0.302975594997406
tensor([[0.2782]])
poem9 of R

The above results shows that poem 1 - Fairyland, 6 - Friend and 3 - Fireflies resonates more with the topics found in the artice scraped from the news.